In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Sep 23 09:45:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#dictionary = {'a':34, 'b':61, 'c':82}
#jsonString = json.dumps(dictionary, indent=4)
#print(jsonString)
#print(bert-base-uncased-3_epochs.0-fold-cv.results.json["f1_macro"])
#test = pd.read_json("bert-base-uncased-3_epochs.0-fold-cv.results.json")
#f = open('bert-base-uncased-3_epochs.0-fold-cv.results.json')
#test = json.load(f)
with open('bert-base-uncased.9-fold-cv.results.json') as json_file:
  data = json.load(json_file)
print(data[0][0]['acc'])
print(data[1][0]['acc'])
print(data[2][0]['acc'])
print(data[3][0]['acc'])
print(data[4][0]['acc'])
print(data[5][0]['acc'])
print(data[6][0]['acc'])
print(data[7][0]['acc'])
print(data[8][0]['acc'])
print(data[9][0]['acc'])

0.9
0.8818181818181818
0.8803030303030303
0.85
0.9028831562974203
0.8998482549317147
0.858877086494689
0.874051593323217
0.8846737481031867
0.8482549317147192


# Setup

## Import Libraries

In [ ]:
# Install FARM & torch
!pip install torch==1.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install farm==0.5.0

# braucht man evtl nicht weils ohne fit() method nicht funktioniert
#!pip install tweet-preprocessor
#!pip install mlxtend
!pip install tweet-preprocessor
# use library below instead
!pip install scipy
import scipy
from scipy import stats

# Here are the imports we need
import torch
from farm.modeling.tokenization import Tokenizer
from farm.data_handler.processor import TextClassificationProcessor
from farm.data_handler.data_silo import DataSilo, DataSiloForCrossVal
from farm.modeling.language_model import LanguageModel
from farm.modeling.prediction_head import TextClassificationHead
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.optimization import initialize_optimizer
from farm.train import Trainer, EarlyStopping
from farm.utils import set_all_seeds, MLFlowLogger, initialize_device_settings
from farm.eval import Evaluator
from farm.evaluation.metrics import simple_accuracy, register_metrics
from transformers import AutoTokenizer
import pandas as pd
from sklearn.metrics import matthews_corrcoef, f1_score
import logging
import numbers
import json
import mlflow
import statistics
from pathlib import Path
from collections import defaultdict
import re
import preprocessor as p
from sklearn.feature_extraction.text import CountVectorizer

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 708.0 MB 10 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.6.0+cu101 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.6.0+cu101 which is incompatible.
     |████████████████████████████████| 207 kB 2.7 MB/s 
     |████████████████████████████████| 2.5 MB 38.6 MB/s 
     |████████████████████████████████| 47.7 MB 76 kB/s 
     |████████████████████████████████| 327 kB 93.9 MB/s 
     |████████████████████████████████| 131 kB 82.5 MB/s 
     |████████████████████████████████| 1.1 MB 6

09/27/2021 07:31:42 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


## Logging with MLflow framework

In [ ]:
# Farm allows simple logging of many parameters & metrics. Let's use the MLflow framework to track our experiment ...
# You will see your results on https://public-mlflow.deepset.ai/
ml_logger = MLFlowLogger(tracking_uri="https://public-mlflow.deepset.ai/")
ml_logger.init_experiment(experiment_name="Public_FARM", run_name="Tutorial1_Colab")


 __          __  _                            _        
 \ \        / / | |                          | |       
  \ \  /\  / /__| | ___ ___  _ __ ___   ___  | |_ ___  
   \ \/  \/ / _ \ |/ __/ _ \| '_ ` _ \ / _ \ | __/ _ \ 
    \  /\  /  __/ | (_| (_) | | | | | |  __/ | || (_) |
     \/  \/ \___|_|\___\___/|_| |_| |_|\___|  \__\___/ 
  ______      _____  __  __  
 |  ____/\   |  __ \|  \/  |              _.-^-._    .--.
 | |__ /  \  | |__) | \  / |           .-'   _   '-. |__|
 |  __/ /\ \ |  _  /| |\/| |          /     |_|     \|  |
 | | / ____ \| | \ \| |  | |         /               \  |
 |_|/_/    \_\_|  \_\_|  |_|        /|     _____     |\ |
                                     |    |==|==|    |  |
|---||---|---|---|---|---|---|---|---|    |--|--|    |  |
|---||---|---|---|---|---|---|---|---|    |==|==|    |  |
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 


## Making sure to fetch the right device

In [ ]:
# We need to fetch the right device to drive the growth of our model
# Make sure that you have gpu turned on in this notebook by going to
# Runtime>Change runtime type and select GPU as Hardware accelerator.

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Devices available: {}".format(device))

Devices available: cuda


## Import Dataset

In [ ]:
# Je nach Dataset müssen wir dieses evtl aufteilen um Trainings- und Testdaten zu erhalten (70/30)

# training data
train = pd.read_table("../content/train.tsv")

# test data without labels to see how our classifiers perform on unlabeled and unseen data
#test = pd.read_csv("./datasets/detecting-insults-in-social-commentary/test.csv")

# dev dataset
dev = pd.read_table("../content/test_with_solutions.tsv")

# test dataset with labels to calculate metrics
test = pd.read_table("../content/impermium_verification_labels.tsv")

## Clean Data

In [ ]:
# remove special characters -> regex schnell und einfach
# set up punctuations we want to be replaced

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [ ]:
# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)
def clean_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr

In [ ]:
# clean training data
train_tweet = clean_tweets(train["Comment"])
train_tweet = pd.DataFrame(train_tweet)

# append cleaned tweets to the training data
train["clean_tweet"] = train_tweet

In [ ]:
# clean the dev data and append the cleaned tweets to the dev data
dev_tweet = clean_tweets(dev["Comment"])
dev_tweet = pd.DataFrame(dev_tweet)
# append cleaned tweets to the training data
dev["clean_tweet"] = dev_tweet

In [ ]:
# clean the test data and append the cleaned tweets to the test data
test_tweet = clean_tweets(test["Comment"])
test_tweet = pd.DataFrame(test_tweet)
# append cleaned tweets to the training data
test["clean_tweet"] = test_tweet

## Extract labels & comments from data

In [ ]:
# extract the labels & comments from the train data

# labels
y_train = train.Insult.values
# comments
X_train = train.clean_tweet.values

# extract the labels & comments from the dev data

# labels
y_dev = dev.Insult.values
# comments
X_dev = dev.clean_tweet.values

# extract the labels & comments from the test data

# labels
y_test = test.Insult.values
# comments
X_test = test.clean_tweet.values

## Vectorize tweets for model building

In [ ]:
# vectorize tweets for model building
vectorizer = CountVectorizer(binary=True, stop_words='english')

# learn a vocabulary dictionary of all tokens in the raw documents
vectorizer.fit(list(X_train) + list(X_dev))

# transform documents to document-term matrix
X_train_vec = vectorizer.transform(X_train)
X_dev_vec = vectorizer.transform(X_dev)
X_test_vec = vectorizer.transform(X_test)

# BERT <br>
https://huggingface.co/bert-base-uncased 

In [ ]:
from transformers import AutoTokenizer
import torch
from farm.modeling.tokenization import Tokenizer
from farm.data_handler.processor import TextClassificationProcessor
from farm.data_handler.data_silo import DataSilo
from farm.modeling.language_model import LanguageModel
from farm.modeling.prediction_head import TextClassificationHead
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.optimization import initialize_optimizer
from farm.train import Trainer
from farm.utils import MLFlowLogger

# Here we initialize a tokenizer that will be used for preprocessing text
# This is the BERT Tokenizer which uses the byte pair encoding method.
# It is currently loaded with a German model

#tokenizer = Tokenizer.load(
#    pretrained_model_name_or_path="finiteautomata/bertweet-base-sentiment-analysis",
#    do_lower_case=False)

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# In order to prepare the data for the model, we need a set of
# functions to transform data files into PyTorch Datasets.
# We group these together in Processor objects.
# We will need a new Processor object for each new source of data.
# The abstract class can be found in farm.data_handling.processor.Processor
# TOXIC = 1
# OTHER = 0
LABEL_LIST = ["0", "1"]
processor = TextClassificationProcessor(tokenizer=tokenizer,
                                        max_seq_len=128,
                                        warmup = 600,
                                        train_filename="train.tsv",
                                        dev_filename="test_with_solutions.tsv",
                                        test_filename="impermium_verification_labels.tsv",
                                        data_dir="../content",
                                        label_list=LABEL_LIST,
                                        metric="f1_macro",
                                        text_column_name="Comment",
                                        label_column_name="Insult")

# We need a DataSilo in order to keep our train, dev and test sets separate.
# The DataSilo will call the functions in the Processor to generate these sets.
# From the DataSilo, we can fetch a PyTorch DataLoader object which will
# be passed on to the model.
# Here is a good place to define a batch size for the model

BATCH_SIZE = 32

data_silo = DataSilo(
    processor=processor,
    batch_size=BATCH_SIZE)

# The language model is the foundation on which modern NLP systems are built.
# They encapsulate a general understanding of sentence semantics
# and are not specific to any one task.

# Here we are using Google's BERT model as implemented by HuggingFace. 
# The model being loaded is a German model that we trained. 
# You can also change the MODEL_NAME_OR_PATH to point to a BERT model that you
# have saved or download one connected to the HuggingFace repository.
# See farm.modeling.language_model.PRETRAINED_MODEL_ARCHIVE_MAP for a list of
# available models

MODEL_NAME_OR_PATH = "bert-base-uncased"
# MODEL_NAME_OR_PATH = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
language_model = LanguageModel.load(MODEL_NAME_OR_PATH)

# A prediction head is a model that processes the output of the language model
# for a specific task.
# Prediction heads will look different depending on whether you're doing text classification
# Named Entity Recognition (NER), question answering or some other task.
# They should generate logits over the available prediction classes and contain methods
# to convert these logits to losses or predictions 

# Here we use TextClassificationHead which receives a single fixed length sentence vector
# and processes it using a feed forward neural network. layer_dims is a list of dimensions:
# [input_dims, hidden_1_dims, hidden_2_dims ..., output_dims]

# Here by default we have a single layer network.
# It takes in a vector of length 768 (the default size of BERT's output).
# It outputs a vector of length 2 (the number of classes in the GermEval18 (coarse) dataset)

prediction_head = TextClassificationHead(num_labels=len(LABEL_LIST), class_weights=data_silo.calculate_class_weights(task_name="text_classification"))

# The language model and prediction head are coupled together in the Adaptive Model.
# This class takes care of model saving and loading and also coordinates
# cases where there is more than one prediction head.

# EMBEDS_DROPOUT_PROB is the probability that an element of the output vector from the
# language model will be set to zero.
# EMBEDS_DROPOUT_PROB = 0.1 distilbert-base-uncased-finetuned-sst-2-english_1.PNG
# EMBEDS_DROPOUT_PROB = 0.01
# EMBEDS_DROPOUT_PROB = 0.2 distilbert-base-uncased-finetuned-sst-2-english_2.PNG
EMBEDS_DROPOUT_PROB = 0.1

model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_sequence"],
    device=device)

# Here we initialize a Bert Adam optimizer that has a linear warmup and warmdown
# Here you can set learning rate, the warmup proportion and number of epochs to train for

#LEARNING_RATE = 2e-5 # distilbert-base-uncased-finetuned-sst-2-english_1.PNG
#LEARNING_RATE = 1e-7
LEARNING_RATE = 1e-5  # distilbert-base-uncased-finetuned-sst-2-english_2.PNG

N_EPOCHS = 3

model, optimizer, lr_schedule = initialize_optimizer(
    model=model,
    device=device,
    learning_rate=LEARNING_RATE,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=N_EPOCHS)

# Training loop handled by this
# It will also trigger evaluation during training using the dev data
# and after training using the test data.

# Set N_GPU to a positive value if CUDA is available
N_GPU = 1

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=N_EPOCHS,
    n_gpu=N_GPU,
    lr_schedule=lr_schedule,
    device=device, 
)

!nvidia-smi

# let the model grow
model = trainer.train()

# Store the model
#save_dir = "save/bert-based-uncased-1"
#model.save(save_dir)
#processor.save(save_dir)

# check it with inference
#basic_texts = [
#    {"text": "Schartau sagte dem Tagesspiegel, dass Fischer ein Idiot ist"},
#    {"text": "Martin Müller spielt Fussball"},
#]
#model = Inferencer(save_dir)
#result = model.inference_from_dicts(dicts=basic_texts)
#print(result)

09/22/2021 09:23:49 - INFO - filelock -   Lock 140467895349072 acquired on /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517.lock


Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

09/22/2021 09:23:50 - INFO - filelock -   Lock 140467895349072 released on /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517.lock
09/22/2021 09:23:50 - INFO - filelock -   Lock 140467895369360 acquired on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

09/22/2021 09:23:50 - INFO - filelock -   Lock 140467895369360 released on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock
09/22/2021 09:23:51 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
09/22/2021 09:23:51 - INFO - farm.data_handler.data_silo -   Loading train set from: ../content/train.tsv 
09/22/2021 09:23:51 - INFO - farm.data_handler.data_silo -   Got ya 1 parallel workers to convert 3947 dictionaries to pytorch datasets (chunksize = 790)...
09/22/2021 09:23:51 - INFO - farm.data_handler.data_silo -    0 
09/22/2021 09:23:51 - INFO - farm.data_handler.data_silo -   /w\
09/22/2021 09:23:51 - INFO - farm.data_handler.data_silo -   / \
09/22/2021 09:23:51 - INFO - farm.data_handler.data_silo 

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

09/22/2021 09:24:26 - INFO - filelock -   Lock 140467889249040 released on /root/.cache/torch/transformers/f2ee78bdd635b758cc0a12352586868bef80e47401abe4c4fcc3832421e7338b.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157.lock
09/22/2021 09:24:28 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
09/22/2021 09:24:28 - INFO - farm.modeling.prediction_head -   Prediction head initialized with size [768, 2]
09/22/2021 09:24:28 - INFO - farm.modeling.prediction_head -   Using class weights for task 'text_classification': [0.6810969 1.8804762]
09/22/2021 09:24:33 - INFO - farm.modeling.optimization -   Loading optimizer `TransformersAdamW`: '{'correct_bias': False, 'weight_decay': 0.01, 'lr': 1e-05}'
09/22/2021 09:24:34 - INFO - farm.modeling.optimization -   Using scheduler 'get_linear_schedule_wi

Wed Sep 22 09:24:36 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W /  70W |   1358MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

09/22/2021 09:24:36 - INFO - farm.train -   
 

          &&& &&  & &&             _____                   _             
      && &\/&\|& ()|/ @, &&       / ____|                 (_)            
      &\/(/&/&||/& /_/)_&/_&     | |  __ _ __ _____      ___ _ __   __ _ 
   &() &\/&|()|/&\/ '%" & ()     | | |_ | '__/ _ \ \ /\ / / | '_ \ / _` |
  &_\_&&_\ |& |&&/&__%_/_& &&    | |__| | | | (_) \ V  V /| | | | | (_| |
&&   && & &| &| /& & % ()& /&&    \_____|_|  \___/ \_/\_/ |_|_| |_|\__, |
 ()&_---()&\&\|&&-&&--%---()~                                       __/ |
     &&     \|||                                                   |___/
             |||
             |||
             |||
       , -=-~  .-^- _
              `

Evaluating: 100%|██████████| 83/83 [00:18<00:00,  4.52it/s]
09/22/2021 09:26:04 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EV

## BERT: k-fold cross validation

In [ ]:
logger = logging.getLogger(__name__)
xval_folds = 10
xval_stratification = True

set_all_seeds(seed=42)
device, n_gpu = initialize_device_settings(use_cuda=True)
n_epochs = 3
batch_size = 32
evaluate_every = 100
dev_split = 0.1
# For xval the dev_stratification parameter must not be None: with None, the devset cannot be created
# using the default method of only splitting by the available chunks as initial train set for each fold
# is just a single chunk!
dev_stratification = True
lang_model = "bert-base-uncased"
do_lower_case = True
use_amp = None

# 1.Create a tokenizer
tokenizer = Tokenizer.load(
    pretrained_model_name_or_path=lang_model,
    do_lower_case=do_lower_case)

def mymetrics(preds, labels):
    acc = simple_accuracy(preds, labels).get("acc")
    f1other = f1_score(y_true=labels, y_pred=preds, pos_label="0")
    f1offense = f1_score(y_true=labels, y_pred=preds, pos_label="1")
    f1macro = f1_score(y_true=labels, y_pred=preds, average="macro")
    f1micro = f1_score(y_true=labels, y_pred=preds, average="micro")
    mcc = matthews_corrcoef(labels, preds)
    return {
        "acc": acc,
        "f1_other": f1other,
        "f1_offense": f1offense,
        "f1_macro": f1macro,
        "f1_micro": f1micro,
        "mcc": mcc
    }
register_metrics('mymetrics', mymetrics)
metric = 'mymetrics'

label_list = ["0", "1"]
processor = TextClassificationProcessor(tokenizer=tokenizer,
                                        max_seq_len=128,
                                        data_dir=Path("../content"),
                                        train_filename="train.tsv",
                                        dev_filename="test_with_solutions.tsv",
                                        test_filename="impermium_verification_labels.tsv",
                                        label_list=label_list,
                                        metric=metric,
                                        dev_split=dev_split,
                                        dev_stratification=dev_stratification,
                                        label_column_name="Insult",
                                        text_column_name="Comment"
                                        )


data_silo = DataSilo(
    processor=processor,
    batch_size=batch_size)

silos = DataSiloForCrossVal.make(data_silo,
                                  sets=["train", "dev"],
                                  n_splits=xval_folds)


def train_on_split(silo_to_use, n_fold, save_dir):
        logger.info(f"############ Crossvalidation: Fold {n_fold} of {xval_folds} ############")
        logger.info(f"Fold training   samples: {len(silo_to_use.data['train'])}")
        logger.info(f"Fold dev        samples: {len(silo_to_use.data['dev'])}")
        logger.info(f"Fold testing    samples: {len(silo_to_use.data['test'])}")
        logger.info( "Total number of samples: "
                    f"{len(silo_to_use.data['train'])+len(silo_to_use.data['dev'])+len(silo_to_use.data['test'])}")

        # Create an AdaptiveModel
        # a) which consists of a pretrained language model as a basis
        language_model = LanguageModel.load(lang_model)
        # b) and a prediction head on top that is suited for our task => Text classification
        prediction_head = TextClassificationHead(
            class_weights=data_silo.calculate_class_weights(task_name="text_classification"),
            num_labels=len(label_list))

        model = AdaptiveModel(
            language_model=language_model,
            prediction_heads=[prediction_head],
            embeds_dropout_prob=0.2,
            lm_output_types=["per_sequence"],
            device=device)

        # Create an optimizer
        model, optimizer, lr_schedule = initialize_optimizer(
            model=model,
            learning_rate=0.5e-5,
            device=device,
            n_batches=len(silo_to_use.loaders["train"]),
            n_epochs=n_epochs,
            use_amp=use_amp)

        # Feed everything to the Trainer, which keeps care of growing our model into powerful plant and evaluates it from time to time
        # Also create an EarlyStopping instance and pass it on to the trainer

        # An early stopping instance can be used to save the model that performs best on the dev set
        # according to some metric and stop training when no improvement is happening for some iterations.
        # NOTE: Using a different save directory for each fold, allows us afterwards to use the
        # nfolds best models in an ensemble!
        save_dir = Path(str(save_dir) + f"-{n_fold}")
        earlystopping = EarlyStopping(
            metric="f1_offense", mode="max",   # use the metric from our own metrics function instead of loss
            save_dir=save_dir,  # where to save the best model
            patience=5    # number of evaluations to wait for improvement before terminating the training
        )

        trainer = Trainer(
            model=model,
            optimizer=optimizer,
            data_silo=silo_to_use,
            epochs=n_epochs,
            n_gpu=n_gpu,
            lr_schedule=lr_schedule,
            evaluate_every=evaluate_every,
            device=device,
            early_stopping=earlystopping,
            evaluator_test=False)

        # train it
        trainer.train()

        return trainer.model

allresults = []
bestfold = None
bestf1_offense = -1
save_dir = Path("saved_models/bert")
for num_fold, silo in enumerate(silos):
    mlflow.start_run(run_name=f"fold-{num_fold + 1}-of-{len(silos)}", nested=True)
    model = train_on_split(silo, num_fold, save_dir)

    # do eval on test set here (and not in Trainer),
    #  so that we can easily store the actual preds and labels for a "global" eval across all folds.
    evaluator_test = Evaluator(
        data_loader=silo.get_data_loader("test"),
        tasks=silo.processor.tasks,
        device=device
    )
    result = evaluator_test.eval(model, return_preds_and_labels=True)
    evaluator_test.log_results(result, "Test", steps=len(silo.get_data_loader("test")), num_fold=num_fold)

    allresults.append(result)

    # keep track of best fold
    f1_offense = result[0]["f1_offense"]
    if f1_offense > bestf1_offense:
        bestf1_offense = f1_offense
        bestfold = num_fold
    mlflow.end_run()
    # emtpy cache to avoid memory leak and cuda OOM across multiple folds
    model.cpu()
    torch.cuda.empty_cache()

# Save the per-fold results to json for a separate, more detailed analysis
with open(f'{lang_model}.all-folds-cv.results.json', 'wt') as fp:
  json.dump(allresults, fp)

09/24/2021 08:21:05 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
09/24/2021 08:21:05 - INFO - filelock -   Lock 139951881552400 acquired on /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517.lock


Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

09/24/2021 08:21:06 - INFO - filelock -   Lock 139951881552400 released on /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517.lock
09/24/2021 08:21:06 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'BertTokenizer'
09/24/2021 08:21:06 - INFO - filelock -   Lock 139951881617360 acquired on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

09/24/2021 08:21:07 - INFO - filelock -   Lock 139951881617360 released on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock
09/24/2021 08:21:08 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
09/24/2021 08:21:08 - INFO - farm.data_handler.data_silo -   Loading train set from: ../content/train.tsv 
09/24/2021 08:21:08 - INFO - farm.data_handler.data_silo -   Got ya 3 parallel workers to convert 3947 dictionaries to pytorch datasets (chunksize = 264)...
09/24/2021 08:21:08 - INFO - farm.data_handler.data_silo -    0    0    0 
09/24/2021 08:21:08 - INFO - farm.data_handler.data_silo -   /w\  /w\  /w\
09/24/2021 08:21:08 - INFO - farm.data_handler.data_silo -   /'\  / \  /'\
09/24/2021 08:21:08 - INFO 

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

09/24/2021 08:21:29 - INFO - filelock -   Lock 139951880342928 released on /root/.cache/torch/transformers/f2ee78bdd635b758cc0a12352586868bef80e47401abe4c4fcc3832421e7338b.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157.lock
09/24/2021 08:21:31 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
09/24/2021 08:21:31 - INFO - farm.modeling.prediction_head -   Prediction head initialized with size [768, 2]
09/24/2021 08:21:31 - INFO - farm.modeling.prediction_head -   Using class weights for task 'text_classification': [0.6810969 1.8804762]
09/24/2021 08:21:35 - INFO - farm.modeling.optimization -   Loading optimizer `TransformersAdamW`: '{'correct_bias': False, 'weight_decay': 0.01, 'lr': 5e-06}'
09/24/2021 08:21:36 - INFO - farm.modeling.optimization -   Using scheduler 'get_linear_schedule_wi

# distilbert-base-uncased-finetuned-sst-2-english <br>
https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english

In [ ]:
# Here we initialize a tokenizer that will be used for preprocessing text
# This is the BERT Tokenizer which uses the byte pair encoding method.
# It is currently loaded with a German model

tokenizer = Tokenizer.load(
    pretrained_model_name_or_path="distilbert-base-uncased-finetuned-sst-2-english",
    do_lower_case=False)


# In order to prepare the data for the model, we need a set of
# functions to transform data files into PyTorch Datasets.
# We group these together in Processor objects.
# We will need a new Processor object for each new source of data.
# The abstract class can be found in farm.data_handling.processor.Processor
# TOXIC = 1
# OTHER = 0
LABEL_LIST = ["0", "1"]
processor = TextClassificationProcessor(tokenizer=tokenizer,
                                        max_seq_len=128,
                                        warmup = 600,
                                        train_filename="train.tsv",
                                        dev_filename="test_with_solutions.tsv",
                                        test_filename="impermium_verification_labels.tsv",
                                        data_dir="../content",
                                        label_list=LABEL_LIST,
                                        metric="f1_macro",
                                        text_column_name="Comment",
                                        label_column_name="Insult")

# We need a DataSilo in order to keep our train, dev and test sets separate.
# The DataSilo will call the functions in the Processor to generate these sets.
# From the DataSilo, we can fetch a PyTorch DataLoader object which will
# be passed on to the model.
# Here is a good place to define a batch size for the model

BATCH_SIZE = 32

data_silo = DataSilo(
    processor=processor,
    batch_size=BATCH_SIZE)

# The language model is the foundation on which modern NLP systems are built.
# They encapsulate a general understanding of sentence semantics
# and are not specific to any one task.

# Here we are using Google's BERT model as implemented by HuggingFace. 
# The model being loaded is a German model that we trained. 
# You can also change the MODEL_NAME_OR_PATH to point to a BERT model that you
# have saved or download one connected to the HuggingFace repository.
# See farm.modeling.language_model.PRETRAINED_MODEL_ARCHIVE_MAP for a list of
# available models

MODEL_NAME_OR_PATH = "distilbert-base-uncased-finetuned-sst-2-english"
# MODEL_NAME_OR_PATH = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
language_model = LanguageModel.load(MODEL_NAME_OR_PATH)

# A prediction head is a model that processes the output of the language model
# for a specific task.
# Prediction heads will look different depending on whether you're doing text classification
# Named Entity Recognition (NER), question answering or some other task.
# They should generate logits over the available prediction classes and contain methods
# to convert these logits to losses or predictions 

# Here we use TextClassificationHead which receives a single fixed length sentence vector
# and processes it using a feed forward neural network. layer_dims is a list of dimensions:
# [input_dims, hidden_1_dims, hidden_2_dims ..., output_dims]

# Here by default we have a single layer network.
# It takes in a vector of length 768 (the default size of BERT's output).
# It outputs a vector of length 2 (the number of classes in the GermEval18 (coarse) dataset)

prediction_head = TextClassificationHead(num_labels=len(LABEL_LIST), class_weights=data_silo.calculate_class_weights(task_name="text_classification"))

# The language model and prediction head are coupled together in the Adaptive Model.
# This class takes care of model saving and loading and also coordinates
# cases where there is more than one prediction head.

# EMBEDS_DROPOUT_PROB is the probability that an element of the output vector from the
# language model will be set to zero.
# EMBEDS_DROPOUT_PROB = 0.1 distilbert-base-uncased-finetuned-sst-2-english_1.PNG
# EMBEDS_DROPOUT_PROB = 0.01
# EMBEDS_DROPOUT_PROB = 0.2 distilbert-base-uncased-finetuned-sst-2-english_2.PNG
EMBEDS_DROPOUT_PROB = 0.1

model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_sequence"],
    device=device)

# Here we initialize a Bert Adam optimizer that has a linear warmup and warmdown
# Here you can set learning rate, the warmup proportion and number of epochs to train for

#LEARNING_RATE = 2e-5 # distilbert-base-uncased-finetuned-sst-2-english_1.PNG
#LEARNING_RATE = 1e-7
LEARNING_RATE = 1e-5  # distilbert-base-uncased-finetuned-sst-2-english_2.PNG

N_EPOCHS = 3

model, optimizer, lr_schedule = initialize_optimizer(
    model=model,
    device=device,
    learning_rate=LEARNING_RATE,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=N_EPOCHS)

# Training loop handled by this
# It will also trigger evaluation during training using the dev data
# and after training using the test data.

# Set N_GPU to a positive value if CUDA is available
N_GPU = 1

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=N_EPOCHS,
    n_gpu=N_GPU,
    lr_schedule=lr_schedule,
    device=device, 
)

## distilbert-base-uncased-finetuned-sst-2-english: k-fold cross validation 

In [ ]:
logger = logging.getLogger(__name__)
xval_folds = 10
xval_stratification = True

set_all_seeds(seed=42)
device, n_gpu = initialize_device_settings(use_cuda=True)
n_epochs = 3
batch_size = 32
evaluate_every = 100
dev_split = 0.1
# For xval the dev_stratification parameter must not be None: with None, the devset cannot be created
# using the default method of only splitting by the available chunks as initial train set for each fold
# is just a single chunk!
dev_stratification = True
lang_model = "distilbert-base-uncased-finetuned-sst-2-english"
do_lower_case = True
use_amp = None

# 1.Create a tokenizer
tokenizer = Tokenizer.load(
    pretrained_model_name_or_path=lang_model,
    do_lower_case=do_lower_case)

def mymetrics(preds, labels):
    acc = simple_accuracy(preds, labels).get("acc")
    f1other = f1_score(y_true=labels, y_pred=preds, pos_label="0")
    f1offense = f1_score(y_true=labels, y_pred=preds, pos_label="1")
    f1macro = f1_score(y_true=labels, y_pred=preds, average="macro")
    f1micro = f1_score(y_true=labels, y_pred=preds, average="micro")
    mcc = matthews_corrcoef(labels, preds)
    return {
        "acc": acc,
        "f1_other": f1other,
        "f1_offense": f1offense,
        "f1_macro": f1macro,
        "f1_micro": f1micro,
        "mcc": mcc
    }
register_metrics('mymetrics', mymetrics)
metric = 'mymetrics'

label_list = ["0", "1"]
processor = TextClassificationProcessor(tokenizer=tokenizer,
                                        max_seq_len=128,
                                        data_dir=Path("../content"),
                                        train_filename="train.tsv",
                                        dev_filename="test_with_solutions.tsv",
                                        test_filename="impermium_verification_labels.tsv",
                                        label_list=label_list,
                                        metric=metric,
                                        dev_split=dev_split,
                                        dev_stratification=dev_stratification,
                                        label_column_name="Insult",
                                        text_column_name="Comment"
                                        )


data_silo = DataSilo(
    processor=processor,
    batch_size=batch_size)

silos = DataSiloForCrossVal.make(data_silo,
                                  sets=["train", "dev"],
                                  n_splits=xval_folds)


def train_on_split(silo_to_use, n_fold, save_dir):
        logger.info(f"############ Crossvalidation: Fold {n_fold} of {xval_folds} ############")
        logger.info(f"Fold training   samples: {len(silo_to_use.data['train'])}")
        logger.info(f"Fold dev        samples: {len(silo_to_use.data['dev'])}")
        logger.info(f"Fold testing    samples: {len(silo_to_use.data['test'])}")
        logger.info( "Total number of samples: "
                    f"{len(silo_to_use.data['train'])+len(silo_to_use.data['dev'])+len(silo_to_use.data['test'])}")

        # Create an AdaptiveModel
        # a) which consists of a pretrained language model as a basis
        language_model = LanguageModel.load(lang_model)
        # b) and a prediction head on top that is suited for our task => Text classification
        prediction_head = TextClassificationHead(
            class_weights=data_silo.calculate_class_weights(task_name="text_classification"),
            num_labels=len(label_list))

        model = AdaptiveModel(
            language_model=language_model,
            prediction_heads=[prediction_head],
            embeds_dropout_prob=0.2,
            lm_output_types=["per_sequence"],
            device=device)

        # Create an optimizer
        model, optimizer, lr_schedule = initialize_optimizer(
            model=model,
            learning_rate=0.5e-5,
            device=device,
            n_batches=len(silo_to_use.loaders["train"]),
            n_epochs=n_epochs,
            use_amp=use_amp)

        # Feed everything to the Trainer, which keeps care of growing our model into powerful plant and evaluates it from time to time
        # Also create an EarlyStopping instance and pass it on to the trainer

        # An early stopping instance can be used to save the model that performs best on the dev set
        # according to some metric and stop training when no improvement is happening for some iterations.
        # NOTE: Using a different save directory for each fold, allows us afterwards to use the
        # nfolds best models in an ensemble!
        save_dir = Path(str(save_dir) + f"-{n_fold}")
        earlystopping = EarlyStopping(
            metric="f1_offense", mode="max",   # use the metric from our own metrics function instead of loss
            save_dir=save_dir,  # where to save the best model
            patience=5    # number of evaluations to wait for improvement before terminating the training
        )

        trainer = Trainer(
            model=model,
            optimizer=optimizer,
            data_silo=silo_to_use,
            epochs=n_epochs,
            n_gpu=n_gpu,
            lr_schedule=lr_schedule,
            evaluate_every=evaluate_every,
            device=device,
            early_stopping=earlystopping,
            evaluator_test=False)

        # train it
        trainer.train()

        return trainer.model

allresults = []
bestfold = None
bestf1_offense = -1
save_dir = Path("saved_models/distilbert-base-uncased-finetuned-sst-2-english")
for num_fold, silo in enumerate(silos):
    mlflow.start_run(run_name=f"fold-{num_fold + 1}-of-{len(silos)}", nested=True)
    model = train_on_split(silo, num_fold, save_dir)

    # do eval on test set here (and not in Trainer),
    #  so that we can easily store the actual preds and labels for a "global" eval across all folds.
    evaluator_test = Evaluator(
        data_loader=silo.get_data_loader("test"),
        tasks=silo.processor.tasks,
        device=device
    )
    result = evaluator_test.eval(model, return_preds_and_labels=True)
    evaluator_test.log_results(result, "Test", steps=len(silo.get_data_loader("test")), num_fold=num_fold)

    allresults.append(result)

    # keep track of best fold
    f1_offense = result[0]["f1_offense"]
    if f1_offense > bestf1_offense:
        bestf1_offense = f1_offense
        bestfold = num_fold
    mlflow.end_run()
    # emtpy cache to avoid memory leak and cuda OOM across multiple folds
    model.cpu()
    torch.cuda.empty_cache()

# Save the per-fold results to json for a separate, more detailed analysis
with open(f'{lang_model}.all-folds-cv.results.json', 'wt') as fp:
  json.dump(allresults, fp)

09/24/2021 09:07:53 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
09/24/2021 09:07:53 - INFO - filelock -   Lock 139951800351376 acquired on /root/.cache/torch/transformers/437d6b3001e14ea1853bcee09a1b2557f230862c5a03d3ebd78a4cdb94a79020.879dd190c02fa5a86beee425ffb5735d066a7b5f2f2127d5d16ce06bba536566.lock


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

09/24/2021 09:07:53 - INFO - filelock -   Lock 139951800351376 released on /root/.cache/torch/transformers/437d6b3001e14ea1853bcee09a1b2557f230862c5a03d3ebd78a4cdb94a79020.879dd190c02fa5a86beee425ffb5735d066a7b5f2f2127d5d16ce06bba536566.lock
09/24/2021 09:07:53 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'DistilBertTokenizer'
09/24/2021 09:07:54 - INFO - filelock -   Lock 139951897624336 acquired on /root/.cache/torch/transformers/cdc3ef7c3d094ab850d6b2a624e8117dc94d62f1fc880f6d1658e797f1b2a1fb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

09/24/2021 09:07:54 - INFO - filelock -   Lock 139951897624336 released on /root/.cache/torch/transformers/cdc3ef7c3d094ab850d6b2a624e8117dc94d62f1fc880f6d1658e797f1b2a1fb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock
09/24/2021 09:07:56 - WARNING - farm.utils -   Failed to log params: INVALID_PARAMETER_VALUE: Changing param value is not allowed. Param with key='tokenizer' was already logged with value='BertTokenizer' for run ID='4b70e1962c3044269b3bd12f923db5fe. Attempted logging new value 'DistilBertTokenizer'.
09/24/2021 09:07:56 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
09/24/2021 09:07:56 - INFO - farm.data_handler.data_silo -   Loading train set from: ../content/train.tsv 
09/24/2021 09:07:56 - INFO - farm.data_handler.data_silo -   Got ya 3 parallel workers to convert 3947 dicti

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

09/24/2021 09:08:16 - INFO - filelock -   Lock 139951879319184 released on /root/.cache/torch/transformers/dd75d79590ca5f697e98574177cada739798baad3b771fd8f500e08f9f8b78cf.461f3160566473d3587f9f4776a5131b1ed527b0d5fccb4b5f06003f457154bc.lock
09/24/2021 09:08:17 - INFO - farm.modeling.language_model -   Automatically detected language from language model name: english
09/24/2021 09:08:17 - INFO - farm.modeling.prediction_head -   Prediction head initialized with size [768, 2]
09/24/2021 09:08:17 - INFO - farm.modeling.prediction_head -   Using class weights for task 'text_classification': [0.6810969 1.8804762]
09/24/2021 09:08:19 - INFO - farm.modeling.optimization -   Loading optimizer `TransformersAdamW`: '{'correct_bias': False, 'weight_decay': 0.01, 'lr': 5e-06}'
09/24/2021 09:08:20 - INFO - farm.modeling.optimization -   Using scheduler 'get_linear_schedule_with_warmup'
09/24/2021 09:08:20 - INFO - farm.modeling.optimization -   Loading schedule `get_linear_schedule_with_warmup`: '

# RoBERTa <br>
https://huggingface.co/roberta-base 

In [ ]:
from transformers import AutoTokenizer
import torch
from farm.modeling.tokenization import Tokenizer
from farm.data_handler.processor import TextClassificationProcessor
from farm.data_handler.data_silo import DataSilo
from farm.modeling.language_model import LanguageModel
from farm.modeling.prediction_head import TextClassificationHead
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.optimization import initialize_optimizer
from farm.train import Trainer
from farm.utils import MLFlowLogger

# Here we initialize a tokenizer that will be used for preprocessing text
# This is the BERT Tokenizer which uses the byte pair encoding method.
# It is currently loaded with a German model

#tokenizer = Tokenizer.load(
#    pretrained_model_name_or_path="finiteautomata/bertweet-base-sentiment-analysis",
#    do_lower_case=False)

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

# In order to prepare the data for the model, we need a set of
# functions to transform data files into PyTorch Datasets.
# We group these together in Processor objects.
# We will need a new Processor object for each new source of data.
# The abstract class can be found in farm.data_handling.processor.Processor
# TOXIC = 1
# OTHER = 0
LABEL_LIST = ["0", "1"]
processor = TextClassificationProcessor(tokenizer=tokenizer,
                                        max_seq_len=128,
                                        warmup = 600,
                                        train_filename="train.tsv",
                                        dev_filename="test_with_solutions.tsv",
                                        test_filename="impermium_verification_labels.tsv",
                                        data_dir="../content",
                                        label_list=LABEL_LIST,
                                        metric="f1_macro",
                                        text_column_name="Comment",
                                        label_column_name="Insult")

# We need a DataSilo in order to keep our train, dev and test sets separate.
# The DataSilo will call the functions in the Processor to generate these sets.
# From the DataSilo, we can fetch a PyTorch DataLoader object which will
# be passed on to the model.
# Here is a good place to define a batch size for the model

BATCH_SIZE = 32

data_silo = DataSilo(
    processor=processor,
    batch_size=BATCH_SIZE)

# The language model is the foundation on which modern NLP systems are built.
# They encapsulate a general understanding of sentence semantics
# and are not specific to any one task.

# Here we are using Google's BERT model as implemented by HuggingFace. 
# The model being loaded is a German model that we trained. 
# You can also change the MODEL_NAME_OR_PATH to point to a BERT model that you
# have saved or download one connected to the HuggingFace repository.
# See farm.modeling.language_model.PRETRAINED_MODEL_ARCHIVE_MAP for a list of
# available models

MODEL_NAME_OR_PATH = "roberta-base"
# MODEL_NAME_OR_PATH = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
language_model = LanguageModel.load(MODEL_NAME_OR_PATH)

# A prediction head is a model that processes the output of the language model
# for a specific task.
# Prediction heads will look different depending on whether you're doing text classification
# Named Entity Recognition (NER), question answering or some other task.
# They should generate logits over the available prediction classes and contain methods
# to convert these logits to losses or predictions 

# Here we use TextClassificationHead which receives a single fixed length sentence vector
# and processes it using a feed forward neural network. layer_dims is a list of dimensions:
# [input_dims, hidden_1_dims, hidden_2_dims ..., output_dims]

# Here by default we have a single layer network.
# It takes in a vector of length 768 (the default size of BERT's output).
# It outputs a vector of length 2 (the number of classes in the GermEval18 (coarse) dataset)

prediction_head = TextClassificationHead(num_labels=len(LABEL_LIST), class_weights=data_silo.calculate_class_weights(task_name="text_classification"))

# The language model and prediction head are coupled together in the Adaptive Model.
# This class takes care of model saving and loading and also coordinates
# cases where there is more than one prediction head.

# EMBEDS_DROPOUT_PROB is the probability that an element of the output vector from the
# language model will be set to zero.
# EMBEDS_DROPOUT_PROB = 0.1 distilbert-base-uncased-finetuned-sst-2-english_1.PNG
# EMBEDS_DROPOUT_PROB = 0.01
# EMBEDS_DROPOUT_PROB = 0.2 distilbert-base-uncased-finetuned-sst-2-english_2.PNG
EMBEDS_DROPOUT_PROB = 0.1

model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_sequence"],
    device=device)

# Here we initialize a Bert Adam optimizer that has a linear warmup and warmdown
# Here you can set learning rate, the warmup proportion and number of epochs to train for

#LEARNING_RATE = 2e-5 # distilbert-base-uncased-finetuned-sst-2-english_1.PNG
#LEARNING_RATE = 1e-7
LEARNING_RATE = 1e-5  # distilbert-base-uncased-finetuned-sst-2-english_2.PNG

N_EPOCHS = 3

model, optimizer, lr_schedule = initialize_optimizer(
    model=model,
    device=device,
    learning_rate=LEARNING_RATE,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=N_EPOCHS)

# Training loop handled by this
# It will also trigger evaluation during training using the dev data
# and after training using the test data.

# Set N_GPU to a positive value if CUDA is available
N_GPU = 1

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=N_EPOCHS,
    n_gpu=N_GPU,
    lr_schedule=lr_schedule,
    device=device, 
)

!nvidia-smi

model = trainer.train()

## RoBERTa: k-fold cross validation

In [ ]:
logger = logging.getLogger(__name__)
xval_folds = 10
xval_stratification = True

set_all_seeds(seed=42)
device, n_gpu = initialize_device_settings(use_cuda=True)
n_epochs = 3
batch_size = 32
evaluate_every = 100
dev_split = 0.1
# For xval the dev_stratification parameter must not be None: with None, the devset cannot be created
# using the default method of only splitting by the available chunks as initial train set for each fold
# is just a single chunk!
dev_stratification = True
lang_model = "roberta-base"
do_lower_case = True
use_amp = None

# 1.Create a tokenizer
tokenizer = Tokenizer.load(
    pretrained_model_name_or_path=lang_model,
    do_lower_case=do_lower_case)

def mymetrics(preds, labels):
    acc = simple_accuracy(preds, labels).get("acc")
    f1other = f1_score(y_true=labels, y_pred=preds, pos_label="0")
    f1offense = f1_score(y_true=labels, y_pred=preds, pos_label="1")
    f1macro = f1_score(y_true=labels, y_pred=preds, average="macro")
    f1micro = f1_score(y_true=labels, y_pred=preds, average="micro")
    mcc = matthews_corrcoef(labels, preds)
    return {
        "acc": acc,
        "f1_other": f1other,
        "f1_offense": f1offense,
        "f1_macro": f1macro,
        "f1_micro": f1micro,
        "mcc": mcc
    }
register_metrics('mymetrics', mymetrics)
metric = 'mymetrics'

label_list = ["0", "1"]
processor = TextClassificationProcessor(tokenizer=tokenizer,
                                        max_seq_len=128,
                                        data_dir=Path("../content"),
                                        train_filename="train.tsv",
                                        dev_filename="test_with_solutions.tsv",
                                        test_filename="impermium_verification_labels.tsv",
                                        label_list=label_list,
                                        metric=metric,
                                        dev_split=dev_split,
                                        dev_stratification=dev_stratification,
                                        label_column_name="Insult",
                                        text_column_name="Comment"
                                        )


data_silo = DataSilo(
    processor=processor,
    batch_size=batch_size)

silos = DataSiloForCrossVal.make(data_silo,
                                  sets=["train", "dev"],
                                  n_splits=xval_folds)


def train_on_split(silo_to_use, n_fold, save_dir):
        logger.info(f"############ Crossvalidation: Fold {n_fold} of {xval_folds} ############")
        logger.info(f"Fold training   samples: {len(silo_to_use.data['train'])}")
        logger.info(f"Fold dev        samples: {len(silo_to_use.data['dev'])}")
        logger.info(f"Fold testing    samples: {len(silo_to_use.data['test'])}")
        logger.info( "Total number of samples: "
                    f"{len(silo_to_use.data['train'])+len(silo_to_use.data['dev'])+len(silo_to_use.data['test'])}")

        # Create an AdaptiveModel
        # a) which consists of a pretrained language model as a basis
        language_model = LanguageModel.load(lang_model)
        # b) and a prediction head on top that is suited for our task => Text classification
        prediction_head = TextClassificationHead(
            class_weights=data_silo.calculate_class_weights(task_name="text_classification"),
            num_labels=len(label_list))

        model = AdaptiveModel(
            language_model=language_model,
            prediction_heads=[prediction_head],
            embeds_dropout_prob=0.2,
            lm_output_types=["per_sequence"],
            device=device)

        # Create an optimizer
        model, optimizer, lr_schedule = initialize_optimizer(
            model=model,
            learning_rate=0.5e-5,
            device=device,
            n_batches=len(silo_to_use.loaders["train"]),
            n_epochs=n_epochs,
            use_amp=use_amp)

        # Feed everything to the Trainer, which keeps care of growing our model into powerful plant and evaluates it from time to time
        # Also create an EarlyStopping instance and pass it on to the trainer

        # An early stopping instance can be used to save the model that performs best on the dev set
        # according to some metric and stop training when no improvement is happening for some iterations.
        # NOTE: Using a different save directory for each fold, allows us afterwards to use the
        # nfolds best models in an ensemble!
        save_dir = Path(str(save_dir) + f"-{n_fold}")
        earlystopping = EarlyStopping(
            metric="f1_offense", mode="max",   # use the metric from our own metrics function instead of loss
            save_dir=save_dir,  # where to save the best model
            patience=5    # number of evaluations to wait for improvement before terminating the training
        )

        trainer = Trainer(
            model=model,
            optimizer=optimizer,
            data_silo=silo_to_use,
            epochs=n_epochs,
            n_gpu=n_gpu,
            lr_schedule=lr_schedule,
            evaluate_every=evaluate_every,
            device=device,
            early_stopping=earlystopping,
            evaluator_test=False)

        # train it
        trainer.train()

        return trainer.model

allresults = []
bestfold = None
bestf1_offense = -1
save_dir = Path("saved_models/roberta-base")
for num_fold, silo in enumerate(silos):
    mlflow.start_run(run_name=f"fold-{num_fold + 1}-of-{len(silos)}", nested=True)
    model = train_on_split(silo, num_fold, save_dir)

    # do eval on test set here (and not in Trainer),
    #  so that we can easily store the actual preds and labels for a "global" eval across all folds.
    evaluator_test = Evaluator(
        data_loader=silo.get_data_loader("test"),
        tasks=silo.processor.tasks,
        device=device
    )
    result = evaluator_test.eval(model, return_preds_and_labels=True)
    evaluator_test.log_results(result, "Test", steps=len(silo.get_data_loader("test")), num_fold=num_fold)

    allresults.append(result)

    # keep track of best fold
    f1_offense = result[0]["f1_offense"]
    if f1_offense > bestf1_offense:
        bestf1_offense = f1_offense
        bestfold = num_fold
    mlflow.end_run()
    # emtpy cache to avoid memory leak and cuda OOM across multiple folds
    model.cpu()
    torch.cuda.empty_cache()

# Save the per-fold results to json for a separate, more detailed analysis
with open(f'{lang_model}.all-folds-cv.results.json', 'wt') as fp:
  json.dump(allresults, fp)

09/27/2021 08:02:56 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
09/27/2021 08:02:57 - INFO - filelock -   Lock 140245060349840 acquired on /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690.lock


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

09/27/2021 08:02:57 - INFO - filelock -   Lock 140245060349840 released on /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690.lock
09/27/2021 08:02:57 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'RobertaTokenizer'
09/27/2021 08:02:58 - INFO - filelock -   Lock 140245059748560 acquired on /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b.lock


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

09/27/2021 08:03:00 - INFO - filelock -   Lock 140245059748560 released on /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b.lock
09/27/2021 08:03:01 - INFO - filelock -   Lock 140245059745040 acquired on /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda.lock


Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

09/27/2021 08:03:02 - INFO - filelock -   Lock 140245059745040 released on /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda.lock
09/27/2021 08:03:04 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
09/27/2021 08:03:04 - INFO - farm.data_handler.data_silo -   Loading train set from: ../content/train.tsv 
09/27/2021 08:03:04 - INFO - farm.data_handler.data_silo -   Got ya 3 parallel workers to convert 3947 dictionaries to pytorch datasets (chunksize = 264)...
09/27/2021 08:03:04 - INFO - farm.data_handler.data_silo -    0    0    0 
09/27/2021 08:03:04 - INFO - farm.data_handler.data_silo -   /w\  /w\  /w\
09/27/2021 08:03:04 - INFO - farm.data_handler.data_silo -   /'\  / \  /'\
09/27/2021 08:03:04 - INFO 

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

09/27/2021 08:03:34 - INFO - filelock -   Lock 140245053120464 released on /root/.cache/torch/transformers/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a121358e1.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e.lock
09/27/2021 08:03:37 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
09/27/2021 08:03:37 - INFO - farm.modeling.prediction_head -   Prediction head initialized with size [768, 2]
09/27/2021 08:03:37 - INFO - farm.modeling.prediction_head -   Using class weights for task 'text_classification': [0.6810969 1.8804762]
09/27/2021 08:03:42 - INFO - farm.modeling.optimization -   Loading optimizer `TransformersAdamW`: '{'correct_bias': False, 'weight_decay': 0.01, 'lr': 5e-06}'
09/27/2021 08:03:44 - INFO - farm.modeling.optimization -   Using scheduler 'get_linear_schedule_wi

# Testing
Because there is a small tendency towards RoBERTa beeing the best performing model, I specifically want to retrieve the best performing fold of RoBERTa to download the model and the processor of the best performing fold for later usage in the tool

In [ ]:
print(bestfold)

5


In [ ]:
# restore model from the best fold
lm_name = model.language_model.name
save_dir = Path(f"saved_models/roberta-base-{bestfold}")
model = AdaptiveModel.load(save_dir, device, lm_name=lm_name)
model.connect_heads_with_processor(data_silo.processor.tasks, require_labels=True)

09/27/2021 09:46:55 - INFO - farm.modeling.adaptive_model -   Found files for loading 1 prediction heads
09/27/2021 09:46:55 - WARNING - farm.modeling.prediction_head -   `layer_dims` will be deprecated in future releases
09/27/2021 09:46:55 - INFO - farm.modeling.prediction_head -   Prediction head initialized with size [768, 2]
09/27/2021 09:46:55 - INFO - farm.modeling.prediction_head -   Using class weights for task 'text_classification': [0.6810969114303589, 1.8804762363433838]
09/27/2021 09:46:55 - INFO - farm.modeling.prediction_head -   Loading prediction head from saved_models/roberta-base-5/prediction_head_0.bin


In [ ]:
# Test your model on a sample (Inference)
from farm.infer import Inferencer
from pprint import PrettyPrinter

infer_model = Inferencer(processor=processor, model=model, task_type="text_classification", gpu=True)

basic_texts = [
    {"text": "Martin is an idiot"},
    {"text": "I think Martin is an idiot"},
    {"text": "fuck you Martin is an idiot"},
    {"text": "Martin Müller spielt Handball in Berlin"},
]
result = infer_model.inference_from_dicts(dicts=basic_texts)
PrettyPrinter().pprint(result)

09/27/2021 10:01:25 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
09/27/2021 10:01:25 - INFO - farm.infer -   Got ya 3 parallel workers to do inference ...
09/27/2021 10:01:25 - INFO - farm.infer -    0    0    0 
09/27/2021 10:01:25 - INFO - farm.infer -   /w\  /w\  /w\
09/27/2021 10:01:25 - INFO - farm.infer -   /'\  / \  /'\
09/27/2021 10:01:25 - INFO - farm.infer -       
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils.py:460: FutureWarning: `is_pretokenized` is deprecated and will be removed in a future version, use `is_split_into_words` instead.
  FutureWarning,
09/27/2021 10:01:25 - INFO - farm.data_handler.processor -   *** Show 2 random examples ***
09/27/2021 10:01:25 - INFO - farm.data_handler.processor -   

      .--.        _____                       _      
    .'_\/_'.     / ____|                     | |     
    '. /\ .'    | (___   __ _ _ __ ___  _ __ | | ___ 
      "||"    

[{'predictions': [{'context': 'Martin is an idiot',
                   'end': None,
                   'label': '1',
                   'probability': 0.9556915,
                   'start': None},
                  {'context': 'I think Martin is an idiot',
                   'end': None,
                   'label': '1',
                   'probability': 0.7768395,
                   'start': None},
                  {'context': 'fuck you Martin is an idiot',
                   'end': None,
                   'label': '1',
                   'probability': 0.9933509,
                   'start': None},
                  {'context': 'Martin Müller spielt Handball in Berlin',
                   'end': None,
                   'label': '0',
                   'probability': 0.9948933,
                   'start': None}],
  'task': 'text_classification'}]


In [ ]:
possible_toxic_comments = []
#result[0]['predictions']
#result[0]['predictions'][0]
result[0]['predictions'][0]['context']
for x in range(len(basic_texts)-1):
  if result[0]['predictions'][x]['label'] == '1':
    #print(result[0]['predictions'][x]['context'])
    possible_toxic_comments.append(result[0]['predictions'][x]['context'])

possible_toxic_comments

['Martin is an idiot',
 'I think Martin is an idiot',
 'fuck you Martin is an idiot']

# ALBERT <br>
https://huggingface.co/albert-base-v2

In [ ]:
from transformers import AutoTokenizer
import torch
from farm.modeling.tokenization import Tokenizer
from farm.data_handler.processor import TextClassificationProcessor
from farm.data_handler.data_silo import DataSilo
from farm.modeling.language_model import LanguageModel
from farm.modeling.prediction_head import TextClassificationHead
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.optimization import initialize_optimizer
from farm.train import Trainer
from farm.utils import MLFlowLogger

# Here we initialize a tokenizer that will be used for preprocessing text
# This is the BERT Tokenizer which uses the byte pair encoding method.
# It is currently loaded with a German model

#tokenizer = Tokenizer.load(
#    pretrained_model_name_or_path="finiteautomata/bertweet-base-sentiment-analysis",
#    do_lower_case=False)

tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")

# In order to prepare the data for the model, we need a set of
# functions to transform data files into PyTorch Datasets.
# We group these together in Processor objects.
# We will need a new Processor object for each new source of data.
# The abstract class can be found in farm.data_handling.processor.Processor
# TOXIC = 1
# OTHER = 0
LABEL_LIST = ["0", "1"]
processor = TextClassificationProcessor(tokenizer=tokenizer,
                                        max_seq_len=128,
                                        warmup = 600,
                                        train_filename="train.tsv",
                                        dev_filename="test_with_solutions.tsv",
                                        test_filename="impermium_verification_labels.tsv",
                                        data_dir="../content",
                                        label_list=LABEL_LIST,
                                        metric="f1_macro",
                                        text_column_name="Comment",
                                        label_column_name="Insult")

# We need a DataSilo in order to keep our train, dev and test sets separate.
# The DataSilo will call the functions in the Processor to generate these sets.
# From the DataSilo, we can fetch a PyTorch DataLoader object which will
# be passed on to the model.
# Here is a good place to define a batch size for the model

BATCH_SIZE = 32

data_silo = DataSilo(
    processor=processor,
    batch_size=BATCH_SIZE)

# The language model is the foundation on which modern NLP systems are built.
# They encapsulate a general understanding of sentence semantics
# and are not specific to any one task.

# Here we are using Google's BERT model as implemented by HuggingFace. 
# The model being loaded is a German model that we trained. 
# You can also change the MODEL_NAME_OR_PATH to point to a BERT model that you
# have saved or download one connected to the HuggingFace repository.
# See farm.modeling.language_model.PRETRAINED_MODEL_ARCHIVE_MAP for a list of
# available models

MODEL_NAME_OR_PATH = "albert-base-v2"
# MODEL_NAME_OR_PATH = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
language_model = LanguageModel.load(MODEL_NAME_OR_PATH)

# A prediction head is a model that processes the output of the language model
# for a specific task.
# Prediction heads will look different depending on whether you're doing text classification
# Named Entity Recognition (NER), question answering or some other task.
# They should generate logits over the available prediction classes and contain methods
# to convert these logits to losses or predictions 

# Here we use TextClassificationHead which receives a single fixed length sentence vector
# and processes it using a feed forward neural network. layer_dims is a list of dimensions:
# [input_dims, hidden_1_dims, hidden_2_dims ..., output_dims]

# Here by default we have a single layer network.
# It takes in a vector of length 768 (the default size of BERT's output).
# It outputs a vector of length 2 (the number of classes in the GermEval18 (coarse) dataset)

prediction_head = TextClassificationHead(num_labels=len(LABEL_LIST), class_weights=data_silo.calculate_class_weights(task_name="text_classification"))

# The language model and prediction head are coupled together in the Adaptive Model.
# This class takes care of model saving and loading and also coordinates
# cases where there is more than one prediction head.

# EMBEDS_DROPOUT_PROB is the probability that an element of the output vector from the
# language model will be set to zero.
# EMBEDS_DROPOUT_PROB = 0.1 distilbert-base-uncased-finetuned-sst-2-english_1.PNG
# EMBEDS_DROPOUT_PROB = 0.01
# EMBEDS_DROPOUT_PROB = 0.2 distilbert-base-uncased-finetuned-sst-2-english_2.PNG
EMBEDS_DROPOUT_PROB = 0.1

model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_sequence"],
    device=device)

# Here we initialize a Bert Adam optimizer that has a linear warmup and warmdown
# Here you can set learning rate, the warmup proportion and number of epochs to train for

#LEARNING_RATE = 2e-5 # distilbert-base-uncased-finetuned-sst-2-english_1.PNG
#LEARNING_RATE = 1e-7
LEARNING_RATE = 1e-5  # distilbert-base-uncased-finetuned-sst-2-english_2.PNG

N_EPOCHS = 3

model, optimizer, lr_schedule = initialize_optimizer(
    model=model,
    device=device,
    learning_rate=LEARNING_RATE,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=N_EPOCHS)

# Training loop handled by this
# It will also trigger evaluation during training using the dev data
# and after training using the test data.

# Set N_GPU to a positive value if CUDA is available
N_GPU = 1

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=N_EPOCHS,
    n_gpu=N_GPU,
    lr_schedule=lr_schedule,
    device=device, 
)

!nvidia-smi

model = trainer.train()

## ALBERT: k-fold cross validation

In [ ]:
logger = logging.getLogger(__name__)
xval_folds = 10
xval_stratification = True

set_all_seeds(seed=42)
device, n_gpu = initialize_device_settings(use_cuda=True)
n_epochs = 3
batch_size = 32
evaluate_every = 100
dev_split = 0.1
# For xval the dev_stratification parameter must not be None: with None, the devset cannot be created
# using the default method of only splitting by the available chunks as initial train set for each fold
# is just a single chunk!
dev_stratification = True
lang_model = "albert-base-v2"
do_lower_case = True
use_amp = None

# 1.Create a tokenizer
tokenizer = Tokenizer.load(
    pretrained_model_name_or_path=lang_model,
    do_lower_case=do_lower_case)

def mymetrics(preds, labels):
    acc = simple_accuracy(preds, labels).get("acc")
    f1other = f1_score(y_true=labels, y_pred=preds, pos_label="0")
    f1offense = f1_score(y_true=labels, y_pred=preds, pos_label="1")
    f1macro = f1_score(y_true=labels, y_pred=preds, average="macro")
    f1micro = f1_score(y_true=labels, y_pred=preds, average="micro")
    mcc = matthews_corrcoef(labels, preds)
    return {
        "acc": acc,
        "f1_other": f1other,
        "f1_offense": f1offense,
        "f1_macro": f1macro,
        "f1_micro": f1micro,
        "mcc": mcc
    }
register_metrics('mymetrics', mymetrics)
metric = 'mymetrics'

label_list = ["0", "1"]
processor = TextClassificationProcessor(tokenizer=tokenizer,
                                        max_seq_len=128,
                                        data_dir=Path("../content"),
                                        train_filename="train.tsv",
                                        dev_filename="test_with_solutions.tsv",
                                        test_filename="impermium_verification_labels.tsv",
                                        label_list=label_list,
                                        metric=metric,
                                        dev_split=dev_split,
                                        dev_stratification=dev_stratification,
                                        label_column_name="Insult",
                                        text_column_name="Comment"
                                        )


data_silo = DataSilo(
    processor=processor,
    batch_size=batch_size)

silos = DataSiloForCrossVal.make(data_silo,
                                  sets=["train", "dev"],
                                  n_splits=xval_folds)


def train_on_split(silo_to_use, n_fold, save_dir):
        logger.info(f"############ Crossvalidation: Fold {n_fold} of {xval_folds} ############")
        logger.info(f"Fold training   samples: {len(silo_to_use.data['train'])}")
        logger.info(f"Fold dev        samples: {len(silo_to_use.data['dev'])}")
        logger.info(f"Fold testing    samples: {len(silo_to_use.data['test'])}")
        logger.info( "Total number of samples: "
                    f"{len(silo_to_use.data['train'])+len(silo_to_use.data['dev'])+len(silo_to_use.data['test'])}")

        # Create an AdaptiveModel
        # a) which consists of a pretrained language model as a basis
        language_model = LanguageModel.load(lang_model)
        # b) and a prediction head on top that is suited for our task => Text classification
        prediction_head = TextClassificationHead(
            class_weights=data_silo.calculate_class_weights(task_name="text_classification"),
            num_labels=len(label_list))

        model = AdaptiveModel(
            language_model=language_model,
            prediction_heads=[prediction_head],
            embeds_dropout_prob=0.2,
            lm_output_types=["per_sequence"],
            device=device)

        # Create an optimizer
        model, optimizer, lr_schedule = initialize_optimizer(
            model=model,
            learning_rate=0.5e-5,
            device=device,
            n_batches=len(silo_to_use.loaders["train"]),
            n_epochs=n_epochs,
            use_amp=use_amp)

        # Feed everything to the Trainer, which keeps care of growing our model into powerful plant and evaluates it from time to time
        # Also create an EarlyStopping instance and pass it on to the trainer

        # An early stopping instance can be used to save the model that performs best on the dev set
        # according to some metric and stop training when no improvement is happening for some iterations.
        # NOTE: Using a different save directory for each fold, allows us afterwards to use the
        # nfolds best models in an ensemble!
        save_dir = Path(str(save_dir) + f"-{n_fold}")
        earlystopping = EarlyStopping(
            metric="f1_offense", mode="max",   # use the metric from our own metrics function instead of loss
            save_dir=save_dir,  # where to save the best model
            patience=5    # number of evaluations to wait for improvement before terminating the training
        )

        trainer = Trainer(
            model=model,
            optimizer=optimizer,
            data_silo=silo_to_use,
            epochs=n_epochs,
            n_gpu=n_gpu,
            lr_schedule=lr_schedule,
            evaluate_every=evaluate_every,
            device=device,
            early_stopping=earlystopping,
            evaluator_test=False)

        # train it
        trainer.train()

        return trainer.model

allresults = []
bestfold = None
bestf1_offense = -1
save_dir = Path("saved_models/albert-base-v2")
for num_fold, silo in enumerate(silos):
    mlflow.start_run(run_name=f"fold-{num_fold + 1}-of-{len(silos)}", nested=True)
    model = train_on_split(silo, num_fold, save_dir)

    # do eval on test set here (and not in Trainer),
    #  so that we can easily store the actual preds and labels for a "global" eval across all folds.
    evaluator_test = Evaluator(
        data_loader=silo.get_data_loader("test"),
        tasks=silo.processor.tasks,
        device=device
    )
    result = evaluator_test.eval(model, return_preds_and_labels=True)
    evaluator_test.log_results(result, "Test", steps=len(silo.get_data_loader("test")), num_fold=num_fold)

    allresults.append(result)

    # keep track of best fold
    f1_offense = result[0]["f1_offense"]
    if f1_offense > bestf1_offense:
        bestf1_offense = f1_offense
        bestfold = num_fold
    mlflow.end_run()
    # emtpy cache to avoid memory leak and cuda OOM across multiple folds
    model.cpu()
    torch.cuda.empty_cache()

# Save the per-fold results to json for a separate, more detailed analysis
with open(f'{lang_model}.all-folds-cv.results.json', 'wt') as fp:
  json.dump(allresults, fp)

09/24/2021 10:24:54 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
09/24/2021 10:24:54 - INFO - filelock -   Lock 139951881619856 acquired on /root/.cache/torch/transformers/0bbb1531ce82f042a813219ffeed7a1fa1f44cd8f78a652c47fc5311e0d40231.978ff53dd976bbf4bc66f09bf4205da0542be753d025263787842df74d15bbca.lock


Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

09/24/2021 10:24:54 - INFO - filelock -   Lock 139951881619856 released on /root/.cache/torch/transformers/0bbb1531ce82f042a813219ffeed7a1fa1f44cd8f78a652c47fc5311e0d40231.978ff53dd976bbf4bc66f09bf4205da0542be753d025263787842df74d15bbca.lock
09/24/2021 10:24:54 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'AlbertTokenizer'
09/24/2021 10:24:54 - INFO - filelock -   Lock 139951887491152 acquired on /root/.cache/torch/transformers/dd1588b85b6fdce1320e224d29ad062e97588e17326b9d05a0b29ee84b8f5f93.c81d4deb77aec08ce575b7a39a989a79dd54f321bfb82c2b54dd35f52f8182cf.lock


Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

09/24/2021 10:24:55 - INFO - filelock -   Lock 139951887491152 released on /root/.cache/torch/transformers/dd1588b85b6fdce1320e224d29ad062e97588e17326b9d05a0b29ee84b8f5f93.c81d4deb77aec08ce575b7a39a989a79dd54f321bfb82c2b54dd35f52f8182cf.lock
09/24/2021 10:24:56 - WARNING - farm.utils -   Failed to log params: INVALID_PARAMETER_VALUE: Changing param value is not allowed. Param with key='tokenizer' was already logged with value='BertTokenizer' for run ID='4b70e1962c3044269b3bd12f923db5fe. Attempted logging new value 'AlbertTokenizer'.
09/24/2021 10:24:56 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
09/24/2021 10:24:56 - INFO - farm.data_handler.data_silo -   Loading train set from: ../content/train.tsv 
09/24/2021 10:24:56 - INFO - farm.data_handler.data_silo -   Got ya 3 parallel workers to convert 3947 dictionar

Downloading:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

09/24/2021 10:25:11 - INFO - filelock -   Lock 139951799412496 released on /root/.cache/torch/transformers/c7c1b2b621933bfa9a5f6ed18b1d6dc2f445001779b13d37286a806117ebeb10.ab806923413c2af99835e13fdbb6014b24af86b0de8edc2d71ef5c646fc54f24.lock
09/24/2021 10:25:11 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
09/24/2021 10:25:11 - INFO - farm.modeling.prediction_head -   Prediction head initialized with size [768, 2]
09/24/2021 10:25:11 - INFO - farm.modeling.prediction_head -   Using class weights for task 'text_classification': [0.6810969 1.8804762]
09/24/2021 10:25:12 - INFO - farm.modeling.optimization -   Loading optimizer `TransformersAdamW`: '{'correct_bias': False, 'weight_decay': 0.01, 'lr': 5e-06}'
09/24/2021 10:25:13 - INFO - farm.modeling.optimization -   Using scheduler 'get_linear_schedule_wi

# DistilBERT <br>
https://huggingface.co/distilbert-base-uncased

In [ ]:
from transformers import AutoTokenizer
import torch
from farm.modeling.tokenization import Tokenizer
from farm.data_handler.processor import TextClassificationProcessor
from farm.data_handler.data_silo import DataSilo
from farm.modeling.language_model import LanguageModel
from farm.modeling.prediction_head import TextClassificationHead
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.optimization import initialize_optimizer
from farm.train import Trainer
from farm.utils import MLFlowLogger

# Here we initialize a tokenizer that will be used for preprocessing text
# This is the BERT Tokenizer which uses the byte pair encoding method.
# It is currently loaded with a German model

#tokenizer = Tokenizer.load(
#    pretrained_model_name_or_path="finiteautomata/bertweet-base-sentiment-analysis",
#    do_lower_case=False)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# In order to prepare the data for the model, we need a set of
# functions to transform data files into PyTorch Datasets.
# We group these together in Processor objects.
# We will need a new Processor object for each new source of data.
# The abstract class can be found in farm.data_handling.processor.Processor
# TOXIC = 1
# OTHER = 0
LABEL_LIST = ["0", "1"]
processor = TextClassificationProcessor(tokenizer=tokenizer,
                                        max_seq_len=128,
                                        warmup = 600,
                                        train_filename="train.tsv",
                                        dev_filename="test_with_solutions.tsv",
                                        test_filename="impermium_verification_labels.tsv",
                                        data_dir="../content",
                                        label_list=LABEL_LIST,
                                        metric="f1_macro",
                                        text_column_name="Comment",
                                        label_column_name="Insult")

# We need a DataSilo in order to keep our train, dev and test sets separate.
# The DataSilo will call the functions in the Processor to generate these sets.
# From the DataSilo, we can fetch a PyTorch DataLoader object which will
# be passed on to the model.
# Here is a good place to define a batch size for the model

BATCH_SIZE = 32

data_silo = DataSilo(
    processor=processor,
    batch_size=BATCH_SIZE)

# The language model is the foundation on which modern NLP systems are built.
# They encapsulate a general understanding of sentence semantics
# and are not specific to any one task.

# Here we are using Google's BERT model as implemented by HuggingFace. 
# The model being loaded is a German model that we trained. 
# You can also change the MODEL_NAME_OR_PATH to point to a BERT model that you
# have saved or download one connected to the HuggingFace repository.
# See farm.modeling.language_model.PRETRAINED_MODEL_ARCHIVE_MAP for a list of
# available models

MODEL_NAME_OR_PATH = "distilbert-base-uncased"
# MODEL_NAME_OR_PATH = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
language_model = LanguageModel.load(MODEL_NAME_OR_PATH)

# A prediction head is a model that processes the output of the language model
# for a specific task.
# Prediction heads will look different depending on whether you're doing text classification
# Named Entity Recognition (NER), question answering or some other task.
# They should generate logits over the available prediction classes and contain methods
# to convert these logits to losses or predictions 

# Here we use TextClassificationHead which receives a single fixed length sentence vector
# and processes it using a feed forward neural network. layer_dims is a list of dimensions:
# [input_dims, hidden_1_dims, hidden_2_dims ..., output_dims]

# Here by default we have a single layer network.
# It takes in a vector of length 768 (the default size of BERT's output).
# It outputs a vector of length 2 (the number of classes in the GermEval18 (coarse) dataset)

prediction_head = TextClassificationHead(num_labels=len(LABEL_LIST), class_weights=data_silo.calculate_class_weights(task_name="text_classification"))

# The language model and prediction head are coupled together in the Adaptive Model.
# This class takes care of model saving and loading and also coordinates
# cases where there is more than one prediction head.

# EMBEDS_DROPOUT_PROB is the probability that an element of the output vector from the
# language model will be set to zero.
# EMBEDS_DROPOUT_PROB = 0.1 distilbert-base-uncased-finetuned-sst-2-english_1.PNG
# EMBEDS_DROPOUT_PROB = 0.01
# EMBEDS_DROPOUT_PROB = 0.2 distilbert-base-uncased-finetuned-sst-2-english_2.PNG
EMBEDS_DROPOUT_PROB = 0.1

model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_sequence"],
    device=device)

# Here we initialize a Bert Adam optimizer that has a linear warmup and warmdown
# Here you can set learning rate, the warmup proportion and number of epochs to train for

#LEARNING_RATE = 2e-5 # distilbert-base-uncased-finetuned-sst-2-english_1.PNG
#LEARNING_RATE = 1e-7
LEARNING_RATE = 1e-5  # distilbert-base-uncased-finetuned-sst-2-english_2.PNG

N_EPOCHS = 3

model, optimizer, lr_schedule = initialize_optimizer(
    model=model,
    device=device,
    learning_rate=LEARNING_RATE,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=N_EPOCHS)

# Training loop handled by this
# It will also trigger evaluation during training using the dev data
# and after training using the test data.

# Set N_GPU to a positive value if CUDA is available
N_GPU = 1

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=N_EPOCHS,
    n_gpu=N_GPU,
    lr_schedule=lr_schedule,
    device=device, 
)

!nvidia-smi

model = trainer.train()

## DistilBERT: k-fold cross validation

In [ ]:
logger = logging.getLogger(__name__)
xval_folds = 10
xval_stratification = True

set_all_seeds(seed=42)
device, n_gpu = initialize_device_settings(use_cuda=True)
n_epochs = 3
batch_size = 32
evaluate_every = 100
dev_split = 0.1
# For xval the dev_stratification parameter must not be None: with None, the devset cannot be created
# using the default method of only splitting by the available chunks as initial train set for each fold
# is just a single chunk!
dev_stratification = True
lang_model = "distilbert-base-uncased"
do_lower_case = True
use_amp = None

# 1.Create a tokenizer
tokenizer = Tokenizer.load(
    pretrained_model_name_or_path=lang_model,
    do_lower_case=do_lower_case)

def mymetrics(preds, labels):
    acc = simple_accuracy(preds, labels).get("acc")
    f1other = f1_score(y_true=labels, y_pred=preds, pos_label="0")
    f1offense = f1_score(y_true=labels, y_pred=preds, pos_label="1")
    f1macro = f1_score(y_true=labels, y_pred=preds, average="macro")
    f1micro = f1_score(y_true=labels, y_pred=preds, average="micro")
    mcc = matthews_corrcoef(labels, preds)
    return {
        "acc": acc,
        "f1_other": f1other,
        "f1_offense": f1offense,
        "f1_macro": f1macro,
        "f1_micro": f1micro,
        "mcc": mcc
    }
register_metrics('mymetrics', mymetrics)
metric = 'mymetrics'

label_list = ["0", "1"]
processor = TextClassificationProcessor(tokenizer=tokenizer,
                                        max_seq_len=128,
                                        data_dir=Path("../content"),
                                        train_filename="train.tsv",
                                        dev_filename="test_with_solutions.tsv",
                                        test_filename="impermium_verification_labels.tsv",
                                        label_list=label_list,
                                        metric=metric,
                                        dev_split=dev_split,
                                        dev_stratification=dev_stratification,
                                        label_column_name="Insult",
                                        text_column_name="Comment"
                                        )


data_silo = DataSilo(
    processor=processor,
    batch_size=batch_size)

silos = DataSiloForCrossVal.make(data_silo,
                                  sets=["train", "dev"],
                                  n_splits=xval_folds)


def train_on_split(silo_to_use, n_fold, save_dir):
        logger.info(f"############ Crossvalidation: Fold {n_fold} of {xval_folds} ############")
        logger.info(f"Fold training   samples: {len(silo_to_use.data['train'])}")
        logger.info(f"Fold dev        samples: {len(silo_to_use.data['dev'])}")
        logger.info(f"Fold testing    samples: {len(silo_to_use.data['test'])}")
        logger.info( "Total number of samples: "
                    f"{len(silo_to_use.data['train'])+len(silo_to_use.data['dev'])+len(silo_to_use.data['test'])}")

        # Create an AdaptiveModel
        # a) which consists of a pretrained language model as a basis
        language_model = LanguageModel.load(lang_model)
        # b) and a prediction head on top that is suited for our task => Text classification
        prediction_head = TextClassificationHead(
            class_weights=data_silo.calculate_class_weights(task_name="text_classification"),
            num_labels=len(label_list))

        model = AdaptiveModel(
            language_model=language_model,
            prediction_heads=[prediction_head],
            embeds_dropout_prob=0.2,
            lm_output_types=["per_sequence"],
            device=device)

        # Create an optimizer
        model, optimizer, lr_schedule = initialize_optimizer(
            model=model,
            learning_rate=0.5e-5,
            device=device,
            n_batches=len(silo_to_use.loaders["train"]),
            n_epochs=n_epochs,
            use_amp=use_amp)

        # Feed everything to the Trainer, which keeps care of growing our model into powerful plant and evaluates it from time to time
        # Also create an EarlyStopping instance and pass it on to the trainer

        # An early stopping instance can be used to save the model that performs best on the dev set
        # according to some metric and stop training when no improvement is happening for some iterations.
        # NOTE: Using a different save directory for each fold, allows us afterwards to use the
        # nfolds best models in an ensemble!
        save_dir = Path(str(save_dir) + f"-{n_fold}")
        earlystopping = EarlyStopping(
            metric="f1_offense", mode="max",   # use the metric from our own metrics function instead of loss
            save_dir=save_dir,  # where to save the best model
            patience=5    # number of evaluations to wait for improvement before terminating the training
        )

        trainer = Trainer(
            model=model,
            optimizer=optimizer,
            data_silo=silo_to_use,
            epochs=n_epochs,
            n_gpu=n_gpu,
            lr_schedule=lr_schedule,
            evaluate_every=evaluate_every,
            device=device,
            early_stopping=earlystopping,
            evaluator_test=False)

        # train it
        trainer.train()

        return trainer.model

allresults = []
bestfold = None
bestf1_offense = -1
save_dir = Path("saved_models/distilbert-base-uncased")
for num_fold, silo in enumerate(silos):
    mlflow.start_run(run_name=f"fold-{num_fold + 1}-of-{len(silos)}", nested=True)
    model = train_on_split(silo, num_fold, save_dir)

    # do eval on test set here (and not in Trainer),
    #  so that we can easily store the actual preds and labels for a "global" eval across all folds.
    evaluator_test = Evaluator(
        data_loader=silo.get_data_loader("test"),
        tasks=silo.processor.tasks,
        device=device
    )
    result = evaluator_test.eval(model, return_preds_and_labels=True)
    evaluator_test.log_results(result, "Test", steps=len(silo.get_data_loader("test")), num_fold=num_fold)

    allresults.append(result)

    # keep track of best fold
    f1_offense = result[0]["f1_offense"]
    if f1_offense > bestf1_offense:
        bestf1_offense = f1_offense
        bestfold = num_fold
    mlflow.end_run()
    # emtpy cache to avoid memory leak and cuda OOM across multiple folds
    model.cpu()
    torch.cuda.empty_cache()

# Save the per-fold results to json for a separate, more detailed analysis
with open(f'{lang_model}.all-folds-cv.results.json', 'wt') as fp:
  json.dump(allresults, fp)

# XLMRoBERTa <br>
https://huggingface.co/xlm-roberta-base 

In [ ]:
from transformers import AutoTokenizer
import torch
from farm.modeling.tokenization import Tokenizer
from farm.data_handler.processor import TextClassificationProcessor
from farm.data_handler.data_silo import DataSilo
from farm.modeling.language_model import LanguageModel
from farm.modeling.prediction_head import TextClassificationHead
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.optimization import initialize_optimizer
from farm.train import Trainer
from farm.utils import MLFlowLogger

# Here we initialize a tokenizer that will be used for preprocessing text
# This is the BERT Tokenizer which uses the byte pair encoding method.
# It is currently loaded with a German model

#tokenizer = Tokenizer.load(
#    pretrained_model_name_or_path="finiteautomata/bertweet-base-sentiment-analysis",
#    do_lower_case=False)

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# In order to prepare the data for the model, we need a set of
# functions to transform data files into PyTorch Datasets.
# We group these together in Processor objects.
# We will need a new Processor object for each new source of data.
# The abstract class can be found in farm.data_handling.processor.Processor
# TOXIC = 1
# OTHER = 0
LABEL_LIST = ["0", "1"]
processor = TextClassificationProcessor(tokenizer=tokenizer,
                                        max_seq_len=128,
                                        warmup = 600,
                                        train_filename="train.tsv",
                                        dev_filename="test_with_solutions.tsv",
                                        test_filename="impermium_verification_labels.tsv",
                                        data_dir="../content",
                                        label_list=LABEL_LIST,
                                        metric="f1_macro",
                                        text_column_name="Comment",
                                        label_column_name="Insult")

# We need a DataSilo in order to keep our train, dev and test sets separate.
# The DataSilo will call the functions in the Processor to generate these sets.
# From the DataSilo, we can fetch a PyTorch DataLoader object which will
# be passed on to the model.
# Here is a good place to define a batch size for the model

BATCH_SIZE = 32

data_silo = DataSilo(
    processor=processor,
    batch_size=BATCH_SIZE)

# The language model is the foundation on which modern NLP systems are built.
# They encapsulate a general understanding of sentence semantics
# and are not specific to any one task.

# Here we are using Google's BERT model as implemented by HuggingFace. 
# The model being loaded is a German model that we trained. 
# You can also change the MODEL_NAME_OR_PATH to point to a BERT model that you
# have saved or download one connected to the HuggingFace repository.
# See farm.modeling.language_model.PRETRAINED_MODEL_ARCHIVE_MAP for a list of
# available models

MODEL_NAME_OR_PATH = "xlm-roberta-base"
# MODEL_NAME_OR_PATH = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
language_model = LanguageModel.load(MODEL_NAME_OR_PATH)

# A prediction head is a model that processes the output of the language model
# for a specific task.
# Prediction heads will look different depending on whether you're doing text classification
# Named Entity Recognition (NER), question answering or some other task.
# They should generate logits over the available prediction classes and contain methods
# to convert these logits to losses or predictions 

# Here we use TextClassificationHead which receives a single fixed length sentence vector
# and processes it using a feed forward neural network. layer_dims is a list of dimensions:
# [input_dims, hidden_1_dims, hidden_2_dims ..., output_dims]

# Here by default we have a single layer network.
# It takes in a vector of length 768 (the default size of BERT's output).
# It outputs a vector of length 2 (the number of classes in the GermEval18 (coarse) dataset)

prediction_head = TextClassificationHead(num_labels=len(LABEL_LIST), class_weights=data_silo.calculate_class_weights(task_name="text_classification"))

# The language model and prediction head are coupled together in the Adaptive Model.
# This class takes care of model saving and loading and also coordinates
# cases where there is more than one prediction head.

# EMBEDS_DROPOUT_PROB is the probability that an element of the output vector from the
# language model will be set to zero.
# EMBEDS_DROPOUT_PROB = 0.1 distilbert-base-uncased-finetuned-sst-2-english_1.PNG
# EMBEDS_DROPOUT_PROB = 0.01
# EMBEDS_DROPOUT_PROB = 0.2 distilbert-base-uncased-finetuned-sst-2-english_2.PNG
EMBEDS_DROPOUT_PROB = 0.1

model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_sequence"],
    device=device)

# Here we initialize a Bert Adam optimizer that has a linear warmup and warmdown
# Here you can set learning rate, the warmup proportion and number of epochs to train for

#LEARNING_RATE = 2e-5 # distilbert-base-uncased-finetuned-sst-2-english_1.PNG
#LEARNING_RATE = 1e-7
LEARNING_RATE = 1e-5  # distilbert-base-uncased-finetuned-sst-2-english_2.PNG

N_EPOCHS = 3

model, optimizer, lr_schedule = initialize_optimizer(
    model=model,
    device=device,
    learning_rate=LEARNING_RATE,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=N_EPOCHS)

# Training loop handled by this
# It will also trigger evaluation during training using the dev data
# and after training using the test data.

# Set N_GPU to a positive value if CUDA is available
N_GPU = 1

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=N_EPOCHS,
    n_gpu=N_GPU,
    lr_schedule=lr_schedule,
    device=device, 
)

!nvidia-smi

model = trainer.train()

## XLMRoBERTa: k-fold cross validation

In [ ]:
logger = logging.getLogger(__name__)
xval_folds = 10
xval_stratification = True

set_all_seeds(seed=42)
device, n_gpu = initialize_device_settings(use_cuda=True)
n_epochs = 3
batch_size = 32
evaluate_every = 100
dev_split = 0.1
# For xval the dev_stratification parameter must not be None: with None, the devset cannot be created
# using the default method of only splitting by the available chunks as initial train set for each fold
# is just a single chunk!
dev_stratification = True
lang_model = "xlm-roberta-base"
do_lower_case = True
use_amp = None

# 1.Create a tokenizer
tokenizer = Tokenizer.load(
    pretrained_model_name_or_path=lang_model,
    do_lower_case=do_lower_case)

def mymetrics(preds, labels):
    acc = simple_accuracy(preds, labels).get("acc")
    f1other = f1_score(y_true=labels, y_pred=preds, pos_label="0")
    f1offense = f1_score(y_true=labels, y_pred=preds, pos_label="1")
    f1macro = f1_score(y_true=labels, y_pred=preds, average="macro")
    f1micro = f1_score(y_true=labels, y_pred=preds, average="micro")
    mcc = matthews_corrcoef(labels, preds)
    return {
        "acc": acc,
        "f1_other": f1other,
        "f1_offense": f1offense,
        "f1_macro": f1macro,
        "f1_micro": f1micro,
        "mcc": mcc
    }
register_metrics('mymetrics', mymetrics)
metric = 'mymetrics'

label_list = ["0", "1"]
processor = TextClassificationProcessor(tokenizer=tokenizer,
                                        max_seq_len=128,
                                        data_dir=Path("../content"),
                                        train_filename="train.tsv",
                                        dev_filename="test_with_solutions.tsv",
                                        test_filename="impermium_verification_labels.tsv",
                                        label_list=label_list,
                                        metric=metric,
                                        dev_split=dev_split,
                                        dev_stratification=dev_stratification,
                                        label_column_name="Insult",
                                        text_column_name="Comment"
                                        )


data_silo = DataSilo(
    processor=processor,
    batch_size=batch_size)

silos = DataSiloForCrossVal.make(data_silo,
                                  sets=["train", "dev"],
                                  n_splits=xval_folds)


def train_on_split(silo_to_use, n_fold, save_dir):
        logger.info(f"############ Crossvalidation: Fold {n_fold} of {xval_folds} ############")
        logger.info(f"Fold training   samples: {len(silo_to_use.data['train'])}")
        logger.info(f"Fold dev        samples: {len(silo_to_use.data['dev'])}")
        logger.info(f"Fold testing    samples: {len(silo_to_use.data['test'])}")
        logger.info( "Total number of samples: "
                    f"{len(silo_to_use.data['train'])+len(silo_to_use.data['dev'])+len(silo_to_use.data['test'])}")

        # Create an AdaptiveModel
        # a) which consists of a pretrained language model as a basis
        language_model = LanguageModel.load(lang_model)
        # b) and a prediction head on top that is suited for our task => Text classification
        prediction_head = TextClassificationHead(
            class_weights=data_silo.calculate_class_weights(task_name="text_classification"),
            num_labels=len(label_list))

        model = AdaptiveModel(
            language_model=language_model,
            prediction_heads=[prediction_head],
            embeds_dropout_prob=0.2,
            lm_output_types=["per_sequence"],
            device=device)

        # Create an optimizer
        model, optimizer, lr_schedule = initialize_optimizer(
            model=model,
            learning_rate=0.5e-5,
            device=device,
            n_batches=len(silo_to_use.loaders["train"]),
            n_epochs=n_epochs,
            use_amp=use_amp)

        # Feed everything to the Trainer, which keeps care of growing our model into powerful plant and evaluates it from time to time
        # Also create an EarlyStopping instance and pass it on to the trainer

        # An early stopping instance can be used to save the model that performs best on the dev set
        # according to some metric and stop training when no improvement is happening for some iterations.
        # NOTE: Using a different save directory for each fold, allows us afterwards to use the
        # nfolds best models in an ensemble!
        save_dir = Path(str(save_dir) + f"-{n_fold}")
        earlystopping = EarlyStopping(
            metric="f1_offense", mode="max",   # use the metric from our own metrics function instead of loss
            save_dir=save_dir,  # where to save the best model
            patience=5    # number of evaluations to wait for improvement before terminating the training
        )

        trainer = Trainer(
            model=model,
            optimizer=optimizer,
            data_silo=silo_to_use,
            epochs=n_epochs,
            n_gpu=n_gpu,
            lr_schedule=lr_schedule,
            evaluate_every=evaluate_every,
            device=device,
            early_stopping=earlystopping,
            evaluator_test=False)

        # train it
        trainer.train()

        return trainer.model

allresults = []
bestfold = None
bestf1_offense = -1
save_dir = Path("saved_models/xlm-roberta-base")
for num_fold, silo in enumerate(silos):
    mlflow.start_run(run_name=f"fold-{num_fold + 1}-of-{len(silos)}", nested=True)
    model = train_on_split(silo, num_fold, save_dir)

    # do eval on test set here (and not in Trainer),
    #  so that we can easily store the actual preds and labels for a "global" eval across all folds.
    evaluator_test = Evaluator(
        data_loader=silo.get_data_loader("test"),
        tasks=silo.processor.tasks,
        device=device
    )
    result = evaluator_test.eval(model, return_preds_and_labels=True)
    evaluator_test.log_results(result, "Test", steps=len(silo.get_data_loader("test")), num_fold=num_fold)

    allresults.append(result)

    # keep track of best fold
    f1_offense = result[0]["f1_offense"]
    if f1_offense > bestf1_offense:
        bestf1_offense = f1_offense
        bestfold = num_fold
    mlflow.end_run()
    # emtpy cache to avoid memory leak and cuda OOM across multiple folds
    model.cpu()
    torch.cuda.empty_cache()

# Save the per-fold results to json for a separate, more detailed analysis
with open(f'{lang_model}.all-folds-cv.results.json', 'wt') as fp:
  json.dump(allresults, fp)

# Comparing classifiers
This section has been moved to the python notebook statistical_analysis.ipynb

## Compare BERT and distilbert-base-uncased-finetuned-sst-2-english using ttest_ind() from scipy

In [ ]:
# create dataframe for t-test

model_name = "bert-base-uncased"

BERT_scores_f1_macro = []
BERT_scores_accuracy = []

for x in range(10):
	string = f"{model_name}.{x}-fold-cv.results.json"
	#print(string)
	with open(string) as json_file:
		  globals()['data%s' % x] = json.load(json_file)


BERT_scores_f1_macro.append(data0[0][0]['f1_macro'])
BERT_scores_accuracy.append(data0[0][0]['acc'])
BERT_scores_f1_macro.append(data1[0][0]['f1_macro'])
BERT_scores_accuracy.append(data1[0][0]['acc'])
BERT_scores_f1_macro.append(data2[0][0]['f1_macro'])
BERT_scores_accuracy.append(data2[0][0]['acc'])
BERT_scores_f1_macro.append(data3[0][0]['f1_macro'])
BERT_scores_accuracy.append(data3[0][0]['acc'])
BERT_scores_f1_macro.append(data4[0][0]['f1_macro'])
BERT_scores_accuracy.append(data4[0][0]['acc'])
BERT_scores_f1_macro.append(data5[0][0]['f1_macro'])
BERT_scores_accuracy.append(data5[0][0]['acc'])
BERT_scores_f1_macro.append(data6[0][0]['f1_macro'])
BERT_scores_accuracy.append(data6[0][0]['acc'])
BERT_scores_f1_macro.append(data7[0][0]['f1_macro'])
BERT_scores_accuracy.append(data7[0][0]['acc'])
BERT_scores_f1_macro.append(data8[0][0]['f1_macro'])
BERT_scores_accuracy.append(data8[0][0]['acc'])
BERT_scores_f1_macro.append(data9[0][0]['f1_macro'])
BERT_scores_accuracy.append(data9[0][0]['acc'])
print(BERT_scores_f1_macro)
print(BERT_scores_accuracy)

folds = [0,1,2,3,4,5,6,7,8,9]
BERT_data = pd.DataFrame(folds, columns=['Fold'])

BERT_data['Model']=f'{model_name}'
BERT_data['F1_macro']=BERT_scores_f1_macro
BERT_data['Accuracy']= BERT_scores_accuracy
BERT_data

#a = [0.8330254524670244, 0, 0, 0, 0]
#b = [0.824509301632236, 0, 0, 0, 0]
#pvalue = stats.ttest_ind(a, b).pvalue





#if pvalue <= 0.05:
#	print('Difference between mean performance is probably real')
#else:
#	print('Algorithms probably have the same performance')

[0.8492875411033978, 0.8492875411033978, 0.8492875411033978, 0.8492875411033978, 0.8492875411033978, 0.8492875411033978, 0.8492875411033978, 0.8492875411033978, 0.8492875411033978, 0.8492875411033978]
[0.8787878787878788, 0.8787878787878788, 0.8787878787878788, 0.8787878787878788, 0.8787878787878788, 0.8787878787878788, 0.8787878787878788, 0.8787878787878788, 0.8787878787878788, 0.8787878787878788]


,Fold,Model,F1_macro,Accuracy
0,0,bert-base-uncased,0.849288,0.878788
1,1,bert-base-uncased,0.849288,0.878788
2,2,bert-base-uncased,0.849288,0.878788
3,3,bert-base-uncased,0.849288,0.878788
4,4,bert-base-uncased,0.849288,0.878788
5,5,bert-base-uncased,0.849288,0.878788
6,6,bert-base-uncased,0.849288,0.878788
7,7,bert-base-uncased,0.849288,0.878788
8,8,bert-base-uncased,0.849288,0.878788
9,9,bert-base-uncased,0.849288,0.878788


In [ ]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

distilbert_base_uncased_finetuned_sst_2_english_scores_f1_macro = []
distilbert_base_uncased_finetuned_sst_2_english_scores_accuracy = []

for x in range(10):
	string = f"{model_name}.{x}-fold-cv.results.json"
	#print(string)
	with open(string) as json_file:
		  globals()['data%s' % x] = json.load(json_file)


distilbert_base_uncased_finetuned_sst_2_english_scores_f1_macro.append(data0[0][0]['f1_macro'])
distilbert_base_uncased_finetuned_sst_2_english_scores_accuracy.append(data0[0][0]['acc'])
distilbert_base_uncased_finetuned_sst_2_english_scores_f1_macro.append(data1[0][0]['f1_macro'])
distilbert_base_uncased_finetuned_sst_2_english_scores_accuracy.append(data1[0][0]['acc'])
distilbert_base_uncased_finetuned_sst_2_english_scores_f1_macro.append(data2[0][0]['f1_macro'])
distilbert_base_uncased_finetuned_sst_2_english_scores_accuracy.append(data2[0][0]['acc'])
distilbert_base_uncased_finetuned_sst_2_english_scores_f1_macro.append(data3[0][0]['f1_macro'])
distilbert_base_uncased_finetuned_sst_2_english_scores_accuracy.append(data3[0][0]['acc'])
distilbert_base_uncased_finetuned_sst_2_english_scores_f1_macro.append(data4[0][0]['f1_macro'])
distilbert_base_uncased_finetuned_sst_2_english_scores_accuracy.append(data4[0][0]['acc'])
distilbert_base_uncased_finetuned_sst_2_english_scores_f1_macro.append(data5[0][0]['f1_macro'])
distilbert_base_uncased_finetuned_sst_2_english_scores_accuracy.append(data5[0][0]['acc'])
distilbert_base_uncased_finetuned_sst_2_english_scores_f1_macro.append(data6[0][0]['f1_macro'])
distilbert_base_uncased_finetuned_sst_2_english_scores_accuracy.append(data6[0][0]['acc'])
distilbert_base_uncased_finetuned_sst_2_english_scores_f1_macro.append(data7[0][0]['f1_macro'])
distilbert_base_uncased_finetuned_sst_2_english_scores_accuracy.append(data7[0][0]['acc'])
distilbert_base_uncased_finetuned_sst_2_english_scores_f1_macro.append(data8[0][0]['f1_macro'])
distilbert_base_uncased_finetuned_sst_2_english_scores_accuracy.append(data8[0][0]['acc'])
distilbert_base_uncased_finetuned_sst_2_english_scores_f1_macro.append(data9[0][0]['f1_macro'])
distilbert_base_uncased_finetuned_sst_2_english_scores_accuracy.append(data9[0][0]['acc'])
print(distilbert_base_uncased_finetuned_sst_2_english_scores_f1_macro)
print(distilbert_base_uncased_finetuned_sst_2_english_scores_accuracy)

folds = [0,1,2,3,4,5,6,7,8,9]
distilbert_base_uncased_finetuned_sst_2_english_data = pd.DataFrame(folds, columns=['Fold'])

distilbert_base_uncased_finetuned_sst_2_english_data['Model']=f'{model_name}'
distilbert_base_uncased_finetuned_sst_2_english_data['F1_macro']=distilbert_base_uncased_finetuned_sst_2_english_scores_f1_macro
distilbert_base_uncased_finetuned_sst_2_english_data['Accuracy']= distilbert_base_uncased_finetuned_sst_2_english_scores_accuracy
distilbert_base_uncased_finetuned_sst_2_english_data

[0.8310600797162394, 0.8310600797162394, 0.8310600797162394, 0.8310600797162394, 0.8310600797162394, 0.8310600797162394, 0.8310600797162394, 0.8310600797162394, 0.8310600797162394, 0.8310600797162394]
[0.8621212121212121, 0.8621212121212121, 0.8621212121212121, 0.8621212121212121, 0.8621212121212121, 0.8621212121212121, 0.8621212121212121, 0.8621212121212121, 0.8621212121212121, 0.8621212121212121]


,Fold,Model,F1_macro,Accuracy
0,0,distilbert-base-uncased-finetuned-sst-2-english,0.83106,0.862121
1,1,distilbert-base-uncased-finetuned-sst-2-english,0.83106,0.862121
2,2,distilbert-base-uncased-finetuned-sst-2-english,0.83106,0.862121
3,3,distilbert-base-uncased-finetuned-sst-2-english,0.83106,0.862121
4,4,distilbert-base-uncased-finetuned-sst-2-english,0.83106,0.862121
5,5,distilbert-base-uncased-finetuned-sst-2-english,0.83106,0.862121
6,6,distilbert-base-uncased-finetuned-sst-2-english,0.83106,0.862121
7,7,distilbert-base-uncased-finetuned-sst-2-english,0.83106,0.862121
8,8,distilbert-base-uncased-finetuned-sst-2-english,0.83106,0.862121
9,9,distilbert-base-uncased-finetuned-sst-2-english,0.83106,0.862121


In [ ]:
# create dataframe for t-test

model_name = "roberta-base"

roberta_base_scores_f1_macro = []
roberta_base_scores_accuracy = []

for x in range(10):
	string = f"{model_name}.{x}-fold-cv.results.json"
	#print(string)
	with open(string) as json_file:
		  globals()['data%s' % x] = json.load(json_file)


roberta_base_scores_f1_macro.append(data0[0][0]['f1_macro'])
roberta_base_scores_accuracy.append(data0[0][0]['acc'])
roberta_base_scores_f1_macro.append(data1[0][0]['f1_macro'])
roberta_base_scores_accuracy.append(data1[0][0]['acc'])
roberta_base_scores_f1_macro.append(data2[0][0]['f1_macro'])
roberta_base_scores_accuracy.append(data2[0][0]['acc'])
roberta_base_scores_f1_macro.append(data3[0][0]['f1_macro'])
roberta_base_scores_accuracy.append(data3[0][0]['acc'])
roberta_base_scores_f1_macro.append(data4[0][0]['f1_macro'])
roberta_base_scores_accuracy.append(data4[0][0]['acc'])
roberta_base_scores_f1_macro.append(data5[0][0]['f1_macro'])
roberta_base_scores_accuracy.append(data5[0][0]['acc'])
roberta_base_scores_f1_macro.append(data6[0][0]['f1_macro'])
roberta_base_scores_accuracy.append(data6[0][0]['acc'])
roberta_base_scores_f1_macro.append(data7[0][0]['f1_macro'])
roberta_base_scores_accuracy.append(data7[0][0]['acc'])
roberta_base_scores_f1_macro.append(data8[0][0]['f1_macro'])
roberta_base_scores_accuracy.append(data8[0][0]['acc'])
roberta_base_scores_f1_macro.append(data9[0][0]['f1_macro'])
roberta_base_scores_accuracy.append(data9[0][0]['acc'])
print(roberta_base_scores_f1_macro)
print(roberta_base_scores_accuracy)

folds = [0,1,2,3,4,5,6,7,8,9]
roberta_base_data = pd.DataFrame(folds, columns=['Fold'])

roberta_base_data['Model']=f'{model_name}'
roberta_base_data['F1_macro']=roberta_base_scores_f1_macro
roberta_base_data['Accuracy']= roberta_base_scores_accuracy
roberta_base_data

In [ ]:
# create dataframe for t-test

model_name = "albert-base-v2"

albert_base_v2_scores_f1_macro = []
albert_base_v2_scores_accuracy = []

for x in range(10):
	string = f"{model_name}.{x}-fold-cv.results.json"
	#print(string)
	with open(string) as json_file:
		  globals()['data%s' % x] = json.load(json_file)


albert_base_v2_scores_f1_macro.append(data0[0][0]['f1_macro'])
albert_base_v2_scores_accuracy.append(data0[0][0]['acc'])
albert_base_v2_scores_f1_macro.append(data1[0][0]['f1_macro'])
albert_base_v2_scores_accuracy.append(data1[0][0]['acc'])
albert_base_v2_scores_f1_macro.append(data2[0][0]['f1_macro'])
albert_base_v2_scores_accuracy.append(data2[0][0]['acc'])
albert_base_v2_scores_f1_macro.append(data3[0][0]['f1_macro'])
albert_base_v2_scores_accuracy.append(data3[0][0]['acc'])
albert_base_v2_scores_f1_macro.append(data4[0][0]['f1_macro'])
albert_base_v2_scores_accuracy.append(data4[0][0]['acc'])
albert_base_v2_scores_f1_macro.append(data5[0][0]['f1_macro'])
albert_base_v2_scores_accuracy.append(data5[0][0]['acc'])
albert_base_v2_scores_f1_macro.append(data6[0][0]['f1_macro'])
albert_base_v2_scores_accuracy.append(data6[0][0]['acc'])
albert_base_v2_scores_f1_macro.append(data7[0][0]['f1_macro'])
albert_base_v2_scores_accuracy.append(data7[0][0]['acc'])
albert_base_v2_scores_f1_macro.append(data8[0][0]['f1_macro'])
albert_base_v2_scores_accuracy.append(data8[0][0]['acc'])
albert_base_v2_scores_f1_macro.append(data9[0][0]['f1_macro'])
albert_base_v2_scores_accuracy.append(data9[0][0]['acc'])
print(albert_base_v2_scores_f1_macro)
print(albert_base_v2_scores_accuracy)

folds = [0,1,2,3,4,5,6,7,8,9]
albert_base_v2_data = pd.DataFrame(folds, columns=['Fold'])

albert_base_v2_data['Model']=f'{model_name}'
albert_base_v2_data['F1_macro']=albert_base_v2_scores_f1_macro
albert_base_v2_data['Accuracy']= albert_base_v2_scores_accuracy
albert_base_v2_data

In [ ]:
# create dataframe for t-test

model_name = "distilbert-base-uncased"

distilbert_base_uncased_scores_f1_macro = []
distilbert_base_uncased_scores_accuracy = []

for x in range(10):
	string = f"{model_name}.{x}-fold-cv.results.json"
	#print(string)
	with open(string) as json_file:
		  globals()['data%s' % x] = json.load(json_file)


distilbert_base_uncased_scores_f1_macro.append(data0[0][0]['f1_macro'])
distilbert_base_uncased_scores_accuracy.append(data0[0][0]['acc'])
distilbert_base_uncased_scores_f1_macro.append(data1[0][0]['f1_macro'])
distilbert_base_uncased_scores_accuracy.append(data1[0][0]['acc'])
distilbert_base_uncased_scores_f1_macro.append(data2[0][0]['f1_macro'])
distilbert_base_uncased_scores_accuracy.append(data2[0][0]['acc'])
distilbert_base_uncased_scores_f1_macro.append(data3[0][0]['f1_macro'])
distilbert_base_uncased_scores_accuracy.append(data3[0][0]['acc'])
distilbert_base_uncased_scores_f1_macro.append(data4[0][0]['f1_macro'])
distilbert_base_uncased_scores_accuracy.append(data4[0][0]['acc'])
distilbert_base_uncased_scores_f1_macro.append(data5[0][0]['f1_macro'])
distilbert_base_uncased_scores_accuracy.append(data5[0][0]['acc'])
distilbert_base_uncased_scores_f1_macro.append(data6[0][0]['f1_macro'])
distilbert_base_uncased_scores_accuracy.append(data6[0][0]['acc'])
distilbert_base_uncased_scores_f1_macro.append(data7[0][0]['f1_macro'])
distilbert_base_uncased_scores_accuracy.append(data7[0][0]['acc'])
distilbert_base_uncased_scores_f1_macro.append(data8[0][0]['f1_macro'])
distilbert_base_uncased_scores_accuracy.append(data8[0][0]['acc'])
distilbert_base_uncased_scores_f1_macro.append(data9[0][0]['f1_macro'])
distilbert_base_uncased_scores_accuracy.append(data9[0][0]['acc'])
print(distilbert_base_uncased_scores_f1_macro)
print(distilbert_base_uncased_scores_accuracy)

folds = [0,1,2,3,4,5,6,7,8,9]
distilbert_base_uncased_data = pd.DataFrame(folds, columns=['Fold'])

distilbert_base_uncased_data['Model']=f'{model_name}'
distilbert_base_uncased_data['F1_macro']=distilbert_base_uncased_scores_f1_macro
distilbert_base_uncased_data['Accuracy']= distilbert_base_uncased_scores_accuracy
distilbert_base_uncased_data

In [ ]:
# create dataframe for t-test

model_name = "xlm-roberta-base"

xlm_roberta_base_scores_f1_macro = []
xlm_roberta_base_scores_accuracy = []

for x in range(10):
	string = f"{model_name}.{x}-fold-cv.results.json"
	#print(string)
	with open(string) as json_file:
		  globals()['data%s' % x] = json.load(json_file)


xlm_roberta_base_scores_f1_macro.append(data0[0][0]['f1_macro'])
xlm_roberta_base_scores_accuracy.append(data0[0][0]['acc'])
xlm_roberta_base_scores_f1_macro.append(data1[0][0]['f1_macro'])
xlm_roberta_base_scores_accuracy.append(data1[0][0]['acc'])
xlm_roberta_base_scores_f1_macro.append(data2[0][0]['f1_macro'])
xlm_roberta_base_scores_accuracy.append(data2[0][0]['acc'])
xlm_roberta_base_scores_f1_macro.append(data3[0][0]['f1_macro'])
xlm_roberta_base_scores_accuracy.append(data3[0][0]['acc'])
xlm_roberta_base_scores_f1_macro.append(data4[0][0]['f1_macro'])
xlm_roberta_base_scores_accuracy.append(data4[0][0]['acc'])
xlm_roberta_base_scores_f1_macro.append(data5[0][0]['f1_macro'])
xlm_roberta_base_scores_accuracy.append(data5[0][0]['acc'])
xlm_roberta_base_scores_f1_macro.append(data6[0][0]['f1_macro'])
xlm_roberta_base_scores_accuracy.append(data6[0][0]['acc'])
xlm_roberta_base_scores_f1_macro.append(data7[0][0]['f1_macro'])
xlm_roberta_base_scores_accuracy.append(data7[0][0]['acc'])
xlm_roberta_base_scores_f1_macro.append(data8[0][0]['f1_macro'])
xlm_roberta_base_scores_accuracy.append(data8[0][0]['acc'])
xlm_roberta_base_scores_f1_macro.append(data9[0][0]['f1_macro'])
xlm_roberta_base_scores_accuracy.append(data9[0][0]['acc'])
print(xlm_roberta_base_scores_f1_macro)
print(xlm_roberta_base_scores_accuracy)

folds = [0,1,2,3,4,5,6,7,8,9]
xlm_roberta_base_data = pd.DataFrame(folds, columns=['Fold'])

xlm_roberta_base_data['Model']=f'{model_name}'
xlm_roberta_base_data['F1_macro']=xlm_roberta_base_scores_f1_macro
xlm_roberta_base_data['Accuracy']= xlm_roberta_base_scores_accuracy
xlm_roberta_base_data

In [ ]:
for x in range(10):
	string = f"bert-base-uncased.{x}-fold-cv.results.json"
	print(string)
	with open(string) as json_file:
		data = json.load(json_file)
	print(data[0][0]['f1_macro'])

bert-base-uncased.0-fold-cv.results.json
0.8492875411033978
bert-base-uncased.1-fold-cv.results.json
0.8492875411033978
bert-base-uncased.2-fold-cv.results.json
0.8492875411033978
bert-base-uncased.3-fold-cv.results.json
0.8492875411033978
bert-base-uncased.4-fold-cv.results.json
0.8492875411033978
bert-base-uncased.5-fold-cv.results.json
0.8492875411033978
bert-base-uncased.6-fold-cv.results.json
0.8492875411033978
bert-base-uncased.7-fold-cv.results.json
0.8492875411033978
bert-base-uncased.8-fold-cv.results.json
0.8492875411033978
bert-base-uncased.9-fold-cv.results.json
0.8492875411033978


In [ ]:
model_name = "bert-base-uncased"



BERT_scores_f1_macro = []
BERT_scores_accuracy = []

for x in range(10):
	string = f"{model_name}.{x}-fold-cv.results.json"
	print(string)
	with open(string) as json_file:
		  globals()['data%s' % x] = json.load(json_file)


BERT_scores_f1_macro.append(data0[0][0]['f1_macro'])
BERT_scores_accuracy.append(data0[0][0]['acc'])
BERT_scores_f1_macro.append(data1[0][0]['f1_macro'])
BERT_scores_accuracy.append(data1[0][0]['acc'])
BERT_scores_f1_macro.append(data2[0][0]['f1_macro'])
BERT_scores_accuracy.append(data2[0][0]['acc'])
BERT_scores_f1_macro.append(data3[0][0]['f1_macro'])
BERT_scores_accuracy.append(data3[0][0]['acc'])
BERT_scores_f1_macro.append(data4[0][0]['f1_macro'])
BERT_scores_accuracy.append(data4[0][0]['acc'])
BERT_scores_f1_macro.append(data5[0][0]['f1_macro'])
BERT_scores_accuracy.append(data5[0][0]['acc'])
BERT_scores_f1_macro.append(data6[0][0]['f1_macro'])
BERT_scores_accuracy.append(data6[0][0]['acc'])
BERT_scores_f1_macro.append(data7[0][0]['f1_macro'])
BERT_scores_accuracy.append(data7[0][0]['acc'])
BERT_scores_f1_macro.append(data8[0][0]['f1_macro'])
BERT_scores_accuracy.append(data8[0][0]['acc'])
BERT_scores_f1_macro.append(data9[0][0]['f1_macro'])
BERT_scores_accuracy.append(data9[0][0]['acc'])

#print(data0)
#print(data9)
#print(BERT_scores_accuracy)

NameError: ignored